Phase 0: Unified bootstrap generator (can create DA or RT selections)
Phase 1a: Single generation preparation (used by both)
Phase 1b: Unified price preparation (handles both DA and RT)
Phase 1c: Unified revenue preparation (handles both DA and RT)
Phase 2: Unified distribution calculator

# Phase 0: Unified bootstrap generator (can create DA or RT selections)

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

class BootstrapSelectionGenerator:
    """
    Phase 0: Generate bootstrap selections for synthetic paths
    Unified implementation for both DAY-AHEAD and REAL-TIME pipelines
    Uses Generation + Price overlap years based on pipeline type
    """
    
    def __init__(self, pipeline_type='da'):
        """
        Initialize the bootstrap generator
        
        Args:
            pipeline_type (str): 'da' for day-ahead or 'rt' for real-time
        """
        # Validate pipeline type
        if pipeline_type not in ['da', 'rt']:
            raise ValueError("pipeline_type must be 'da' or 'rt'")
        
        self.pipeline_type = pipeline_type
        
        # Define paths
        self.data_path = Path('aamani_data')
        self.output_path = Path('Renewable Portfolio LLC')
        
        # Set pipeline-specific configurations
        if pipeline_type == 'da':
            self.bootstrap_folder = self.output_path / 'bootstrap_selections_da'
            self.price_column_primary = 'price_da'
            self.pipeline_name = "DAY-AHEAD"
            self.pipeline_name_short = "DA"
        else:  # rt
            self.bootstrap_folder = self.output_path / 'bootstrap_selections_rt'
            self.price_column_primary = 'price_rt'
            self.pipeline_name = "REAL-TIME"
            self.pipeline_name_short = "RT"
        
        self.bootstrap_folder.mkdir(parents=True, exist_ok=True)
        
        # Get available combined files
        self.available_files = list(self.data_path.glob('*_generation_price_combined.csv'))
        self.available_sites = [f.stem.replace('_generation_price_combined', '') for f in self.available_files]
        
        # Number of synthetic paths to generate
        self.n_synthetic_paths = 10  # Default
        
        # Month names for display
        self.month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        
        # For RT, we need to track which price column to use for each site
        self.site_price_map = {}
    
    def get_site_selection(self):
        """
        Interactive site selection with option for all sites
        """
        print("\n" + "="*60)
        print(f"{self.pipeline_name} BOOTSTRAP SELECTION GENERATOR - SITE SELECTION")
        print("="*60)
        
        if not self.available_sites:
            print("❌ No combined generation-price files found in aamani_data!")
            return None
        
        # Filter sites that have the appropriate price data
        print(f"\n🔍 Checking sites for {self.pipeline_name.lower()} price data...")
        sites_with_price = []
        
        for site in self.available_sites:
            file_path = self.data_path / f"{site}_generation_price_combined.csv"
            print(f"   Checking {site}...", end='')
            
            try:
                # Check if appropriate price column exists
                df_header = pd.read_csv(file_path, nrows=5)
                
                if self.pipeline_type == 'da':
                    # For DA, only check price_da
                    if 'price_da' not in df_header.columns:
                        print(" ✗ (no price_da column)")
                        continue
                    price_col_to_check = 'price_da'
                else:  # rt
                    # For RT, check price_rt first, then price
                    if 'price_rt' in df_header.columns:
                        price_col_to_check = 'price_rt'
                    elif 'price' in df_header.columns:
                        price_col_to_check = 'price'
                        print(f" (using 'price' column for RT)", end='')
                    else:
                        print(" ✗ (no price_rt or price column)")
                        continue
                
                # Check if there's actual price data
                has_price_data = False
                chunk_size = 10000
                for chunk in pd.read_csv(file_path, chunksize=chunk_size, usecols=['year', price_col_to_check]):
                    if chunk[price_col_to_check].notna().any():
                        has_price_data = True
                        first_year = chunk[chunk[price_col_to_check].notna()]['year'].min()
                        print(f" ✓ ({self.pipeline_name_short} prices found, starting {int(first_year)})")
                        break
                
                if has_price_data:
                    sites_with_price.append(site)
                    if self.pipeline_type == 'rt':
                        self.site_price_map[site] = price_col_to_check
                else:
                    print(f" ✗ ({price_col_to_check} column exists but no data)")
                    
            except Exception as e:
                print(f" ✗ (error: {str(e)})")
        
        if not sites_with_price:
            print(f"\n❌ No sites with {self.pipeline_name.lower()} price data found!")
            if self.pipeline_type == 'da':
                print("   Note: Checked for 'price_da' column")
            else:
                print("   Note: Checked for both 'price_rt' and 'price' columns")
            return None
        
        print(f"\n✅ Found {len(sites_with_price)} sites with {self.pipeline_name.lower()} price data")
        print("\nAvailable options:")
        print(f"0. ALL SITES (Generate selections for all sites with {self.pipeline_name_short} prices)")
        for i, site in enumerate(sites_with_price):
            print(f"{i+1}. {site}")
        
        print("="*60)
        
        while True:
            try:
                selection = input("\n📊 Select option number (0 for all sites): ").strip()
                if selection == '0':
                    return 'ALL_SITES', sites_with_price
                else:
                    idx = int(selection) - 1
                    if 0 <= idx < len(sites_with_price):
                        return sites_with_price[idx], sites_with_price
                    else:
                        print("❌ Invalid selection!")
            except:
                print("❌ Please enter a valid number!")
    
    def analyze_data_availability(self, site_name):
        """
        Analyze which years have both generation and price data
        """
        print(f"\n📊 Analyzing {self.pipeline_name} data availability for {site_name}...")
        
        # Load the combined file
        file_path = self.data_path / f"{site_name}_generation_price_combined.csv"
        
        print("   Loading data...")
        
        try:
            # Determine which price column to use
            if self.pipeline_type == 'da':
                price_column = 'price_da'
            else:  # rt
                price_column = self.site_price_map.get(site_name, 'price_rt')
            
            # Check if price column exists
            df_header = pd.read_csv(file_path, nrows=1)
            if price_column not in df_header.columns:
                print(f"   ⚠️  {self.pipeline_name} price column '{price_column}' not found!")
                return None
            
            # Read only necessary columns
            cols_to_read = ['year', 'month', 'generation_mw', price_column]
            df = pd.read_csv(file_path, usecols=cols_to_read)
            
        except Exception as e:
            print(f"   ⚠️  Error reading file: {str(e)}")
            return None
        
        # Find years with generation data
        gen_years = set(int(year) for year in df[df['generation_mw'].notna()]['year'].unique())
        
        # Find years with price data
        price_years = set(int(year) for year in df[df[price_column].notna()]['year'].unique())
        
        # Report findings
        if gen_years:
            print(f"\n   📈 Generation data: {min(gen_years)}-{max(gen_years)} ({len(gen_years)} years)")
        else:
            print("\n   ⚠️  No generation data found!")
            
        if price_years:
            print(f"   💰 {self.pipeline_name} price data: {min(price_years)}-{max(price_years)} ({len(price_years)} years)")
        else:
            print(f"   ⚠️  No {self.pipeline_name.lower()} price data found!")
            return None
        
        # Find overlapping years
        overlap_years = gen_years & price_years
        
        print(f"\n   🔄 Overlapping years (Gen + {self.pipeline_name_short} price): {len(overlap_years)} years")
        if overlap_years:
            overlap_list = sorted(list(overlap_years))
            print(f"      Years: {overlap_list[0]}-{overlap_list[-1]}")
            print(f"      Using these {len(overlap_years)} years for {self.pipeline_name} bootstrap")
        
        return overlap_years
    
    def check_monthly_completeness(self, site_name, years_to_use):
        """
        Check which year-month combinations have complete data
        """
        print(f"\n🔍 Checking monthly data completeness for {self.pipeline_name} pipeline...")
        
        file_path = self.data_path / f"{site_name}_generation_price_combined.csv"
        
        try:
            # Determine which price column to use
            if self.pipeline_type == 'da':
                price_column = 'price_da'
            else:  # rt
                price_column = self.site_price_map.get(site_name, 'price_rt')
            
            # Read only the necessary columns and overlapping years
            cols_to_read = ['year', 'month', 'generation_mw', price_column]
            df = pd.read_csv(file_path, usecols=cols_to_read)
            df = df[df['year'].isin(years_to_use)]
            
            # Check completeness for each year-month
            complete_months = {}
            
            for month in range(1, 13):
                complete_months[month] = []
                
                for year in sorted(years_to_use):
                    month_data = df[(df['year'] == year) & (df['month'] == month)]
                    
                    if len(month_data) > 0:
                        # Check if both generation and price data exist
                        has_gen = month_data['generation_mw'].notna().any()
                        has_price = month_data[price_column].notna().any()
                        
                        if has_gen and has_price:
                            complete_months[month].append(int(year))
            
            # Report findings
            print(f"\n   Monthly data availability (Gen + {self.pipeline_name_short}):")
            for month in range(1, 13):
                years_available = len(complete_months[month])
                print(f"   {self.month_names[month]:3s}: {years_available} years available")
            
            return complete_months
            
        except Exception as e:
            print(f"\n   ⚠️  Error checking monthly completeness: {str(e)}")
            return {month: [] for month in range(1, 13)}
    
    def generate_bootstrap_selections(self, complete_months, n_paths):
        """
        Generate bootstrap selections using monthly blocks
        """
        print(f"\n🎲 Generating {n_paths} {self.pipeline_name} bootstrap selections...")
        
        selections = {}
        
        for path_num in range(1, n_paths + 1):
            path_selection = {}
            
            for month in range(1, 13):
                available_years = complete_months[month]
                
                if available_years:
                    # Randomly select a year for this month
                    selected_year = np.random.choice(available_years)
                    path_selection[self.month_names[month]] = int(selected_year)
                else:
                    # No data available for this month
                    path_selection[self.month_names[month]] = None
                    print(f"   ⚠️  Warning: No {self.pipeline_name_short} data available for {self.month_names[month]}")
            
            selections[f'path_{path_num}'] = path_selection
        
        return selections
    
    def save_selections(self, selections, site_name, metadata):
        """
        Save bootstrap selections to JSON file
        """
        # Create filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{site_name}_bootstrap_selections_{self.pipeline_type}_{timestamp}.json"
        filepath = self.bootstrap_folder / filename
        
        # Prepare output data
        output = {
            'pipeline_type': 'day_ahead' if self.pipeline_type == 'da' else 'real_time',
            'site_name': site_name,
            'generated_at': datetime.now().isoformat(),
            'n_paths': len(selections),
            'metadata': metadata,
            'selections': selections
        }
        
        # Save to JSON
        with open(filepath, 'w') as f:
            json.dump(output, f, indent=2)
        
        print(f"\n💾 Saved: {filepath}")
        
        # Also save a "latest" version for easy access
        latest_filepath = self.bootstrap_folder / f"{site_name}_bootstrap_selections_{self.pipeline_type}_latest.json"
        with open(latest_filepath, 'w') as f:
            json.dump(output, f, indent=2)
        
        print(f"💾 Saved: {latest_filepath} (latest {self.pipeline_name_short} version)")
        
        return filepath
    
    def print_sample_selections(self, selections):
        """
        Print a sample of the selections for verification
        """
        print("\n" + "="*40)
        print(f"SAMPLE {self.pipeline_name} BOOTSTRAP SELECTIONS")
        print("="*40)
        
        # Show first 3 paths
        for path_name in list(selections.keys())[:3]:
            print(f"\n{path_name}:")
            path_data = selections[path_name]
            
            # Show first 6 months
            for month in range(1, 7):
                month_name = self.month_names[month]
                year = path_data.get(month_name, 'N/A')
                print(f"   {month_name}: {year}")
            print("   ...")
    
    def process_single_site(self, site_name):
        """
        Process bootstrap generation for a single site
        """
        print(f"\n{'='*60}")
        print(f"Processing {self.pipeline_name} bootstrap for: {site_name}")
        print(f"{'='*60}")
        
        try:
            # Analyze data availability
            overlap_years = self.analyze_data_availability(site_name)
            
            if not overlap_years or len(overlap_years) < 5:
                print(f"\n❌ Insufficient overlapping years for {self.pipeline_name} (need at least 5)")
                return False
            
            years_list = sorted(list(overlap_years))
            print(f"\n✅ Using {len(overlap_years)} years with generation + {self.pipeline_name.lower()} price")
            print(f"   Years: {years_list[0]}-{years_list[-1]}")
            
            # Check monthly completeness
            complete_months = self.check_monthly_completeness(site_name, overlap_years)
            
            # Generate bootstrap selections
            selections = self.generate_bootstrap_selections(complete_months, self.n_synthetic_paths)
            
            # Print sample
            self.print_sample_selections(selections)
            
            # Prepare metadata
            metadata = {
                'years_used': [int(year) for year in years_list],
                'n_years': len(years_list),
                'price_type': self.pipeline_type,
                'year_range': f"{years_list[0]}-{years_list[-1]}"
            }
            
            # Add price column info for RT
            if self.pipeline_type == 'rt':
                metadata['price_column'] = self.site_price_map.get(site_name, 'price_rt')
            
            # Save selections
            self.save_selections(selections, site_name, metadata)
            
            return True
            
        except Exception as e:
            print(f"\n❌ Error processing {site_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            return False
    
    def run_generator(self):
        """
        Main function to run the bootstrap selection generator
        """
        print(f"\n🎯 {self.pipeline_name} Bootstrap Selection Generator - Phase 0-{self.pipeline_name_short}")
        print(f"   (Generates monthly block selections for {self.pipeline_name_short} price synthetic paths)")
        print("="*60)
        
        # Get site selection
        site_selection = self.get_site_selection()
        if not site_selection:
            print("\n💡 Troubleshooting tips:")
            if self.pipeline_type == 'da':
                print("   1. Ensure your files have 'price_da' column")
                print("   2. Check that DA prices have actual values (not just empty column)")
                print("   3. DA prices typically start around 2010-2015 in most markets")
            else:  # rt
                print("   1. Ensure your files have 'price_rt' or 'price' column")
                print("   2. Check that RT prices have actual values (not just empty column)")
                print("   3. RT prices typically start around 2015-2020 in most markets")
            print("   4. File names should match pattern: *_generation_price_combined.csv")
            return
        
        selection_type, available_sites = site_selection
        
        # Ask for number of synthetic paths
        while True:
            try:
                n_paths = input(f"\n🎲 Number of synthetic paths to generate (default={self.n_synthetic_paths}): ").strip()
                if n_paths == '':
                    break
                else:
                    self.n_synthetic_paths = int(n_paths)
                    if self.n_synthetic_paths > 0:
                        break
                    else:
                        print("❌ Please enter a positive number!")
            except:
                print("❌ Please enter a valid number!")
        
        # Set random seed for reproducibility
        seed = input("\n🌱 Random seed (press Enter for random): ").strip()
        if seed:
            np.random.seed(int(seed))
            print(f"   Using seed: {seed}")
        else:
            random_seed = np.random.randint(0, 100000)
            np.random.seed(random_seed)
            print(f"   Using random seed: {random_seed}")
        
        # Process based on selection
        if selection_type == 'ALL_SITES':
            print("\n" + "="*60)
            print(f"🚀 PROCESSING ALL SITES FOR {self.pipeline_name}")
            print("="*60)
            
            successful = 0
            failed = 0
            
            for i, site_name in enumerate(available_sites, 1):
                print(f"\n[{i}/{len(available_sites)}] Processing {site_name}...")
                
                if self.process_single_site(site_name):
                    successful += 1
                else:
                    failed += 1
            
            # Summary
            print("\n" + "="*60)
            print(f"✨ {self.pipeline_name} BOOTSTRAP GENERATION COMPLETE!")
            print("="*60)
            print(f"\n📊 Summary:")
            print(f"   ✅ Successfully processed: {successful} sites")
            if failed > 0:
                print(f"   ❌ Failed/Skipped: {failed} sites")
            
            print(f"\n📁 {self.pipeline_name} bootstrap selections saved in:")
            print(f"   Renewable Portfolio LLC/bootstrap_selections_{self.pipeline_type}/")
            
        else:
            # Process single site
            if self.process_single_site(selection_type):
                print("\n" + "="*60)
                print(f"✨ {self.pipeline_name} BOOTSTRAP GENERATION COMPLETE!")
                print("="*60)
                print(f"\n{self.pipeline_name} bootstrap selections created for {selection_type}")
                print(f"Number of paths: {self.n_synthetic_paths}")
                
                print(f"\n📁 Files saved in:")
                print(f"   Renewable Portfolio LLC/bootstrap_selections_{self.pipeline_type}/")
                print(f"   - {selection_type}_bootstrap_selections_{self.pipeline_type}_[timestamp].json")
                print(f"   - {selection_type}_bootstrap_selections_{self.pipeline_type}_latest.json")
        
        print(f"\n📌 Next steps for {self.pipeline_name} pipeline:")
        print(f"   1. Run Phase 1a: Generation data preparation")
        print(f"   2. Run Phase 1b: {self.pipeline_name} price data preparation")
        if self.pipeline_type == 'da':
            print(f"   3. Run Phase 1c: Day-Ahead Revenue Calculation")
            print(f"   4. Run Phase 2: Calculate distributions")
        else:
            print(f"   3. Run Phase 1c: Real-Time Revenue Calculation")
            print(f"   4. Run Phase 2: Calculate distributions")
        
        # Ask if user wants to generate for another site
        another = input(f"\n🔄 Generate {self.pipeline_name} bootstrap selections for another site? (y/n): ").strip().lower()
        if another == 'y':
            self.run_generator()

# Example usage
if __name__ == "__main__":
    # Ask user which pipeline to run
    print("\n🌟 Bootstrap Selection Generator")
    print("Select pipeline type:")
    print("1. Day-Ahead (DA)")
    print("2. Real-Time (RT)")
    
    while True:
        choice = input("\nEnter your choice (1 or 2): ").strip()
        if choice == '1':
            generator = BootstrapSelectionGenerator(pipeline_type='da')
            break
        elif choice == '2':
            generator = BootstrapSelectionGenerator(pipeline_type='rt')
            break
        else:
            print("❌ Invalid choice! Please enter 1 or 2.")
    
    generator.run_generator()


🌟 Bootstrap Selection Generator
Select pipeline type:
1. Day-Ahead (DA)
2. Real-Time (RT)



🎯 REAL-TIME Bootstrap Selection Generator - Phase 0-RT
   (Generates monthly block selections for RT price synthetic paths)

REAL-TIME BOOTSTRAP SELECTION GENERATOR - SITE SELECTION

🔍 Checking sites for real-time price data...
   Checking Albemarle_Beach_Solar... ✓ (RT prices found, starting 2012)
   Checking Blue_Wing_Solar_Energy_Generator... ✓ (RT prices found, starting 2012)
   Checking Lamesa_Solar... ✓ (RT prices found, starting 2016)
   Checking Midway_Solar_Farm_III... ✓ (RT prices found, starting 2012)
   Checking Misae_Solar... ✓ (RT prices found, starting 2012)
   Checking Mount_Signal_Solar_Farm_II... ✓ (RT prices found, starting 2012)
   Checking RE_Mustang_LLC... ✓ (RT prices found, starting 2012)

✅ Found 7 sites with real-time price data

Available options:
0. ALL SITES (Generate selections for all sites with RT prices)
1. Albemarle_Beach_Solar
2. Blue_Wing_Solar_Energy_Generator
3. Lamesa_Solar
4. Midway_Solar_Farm_III
5. Misae_Solar
6. Mount_Signal_Solar_Farm_II
7. 

# Phase 1a: Single generation preparation (used by both)

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

class GenerationDataPreparation:
    """
    Phase 1a: Create historical data and timeseries (with synthetic paths)
    SHARED implementation for both DA and RT pipelines
    Since overlapping years are similar, generation data is common to both
    Will use whichever bootstrap exists (DA or RT), preferring DA if both exist
    """
    
    def __init__(self):
        """
        Initialize the generation data preparation
        No pipeline_type needed - generation is shared!
        """
        
        # Define paths
        self.data_path = Path('aamani_data')
        self.base_output_path = Path('Renewable Portfolio LLC')
        
        # Bootstrap paths - will check both
        self.bootstrap_path_da = self.base_output_path / 'bootstrap_selections_da'
        self.bootstrap_path_rt = self.base_output_path / 'bootstrap_selections_rt'
        
        # Will be set when loading bootstrap
        self.bootstrap_type = None  # 'da' or 'rt'
        self.pipeline_name = None
        self.pipeline_name_short = None
        
        # Get available combined files
        self.available_files = list(self.data_path.glob('*_generation_price_combined.csv'))
        self.available_sites = [f.stem.replace('_generation_price_combined', '') for f in self.available_files]
        
        # Bootstrap selections will be loaded per site
        self.bootstrap_selections = None
        self.n_synthetic_paths = None
        self.bootstrap_metadata = None
        
        # Month names for labeling
        self.month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        self.month_names_full = ['', 'January', 'February', 'March', 'April', 'May', 'June',
                                'July', 'August', 'September', 'October', 'November', 'December']
        
        # Define weather columns that should be averaged (not summed)
        self.weather_cols_avg = [
            'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'cloud_cover',
            'surface_pressure', 'wind_speed_100m', 'wind_speed_10m',
            'shortwave_radiation', 'diffuse_radiation', 'direct_normal_irradiance'
        ]
        
        # Define weather columns that should be summed
        self.weather_cols_sum = ['precipitation']
        
        # Wind direction columns to skip in daily/monthly aggregations
        self.weather_cols_skip = ['wind_direction_100m', 'wind_direction_10m']
    
    def get_site_selection(self):
        """
        Interactive site selection with option for all sites
        Checks for either DA or RT bootstrap selections
        """
        print("\n" + "="*60)
        print("GENERATION DATA PREPARATION - SITE SELECTION")
        print("(Shared for both DA and RT pipelines)")
        print("="*60)
        
        if not self.available_sites:
            print("❌ No combined generation-price files found in aamani_data!")
            return None
        
        # Check which sites have bootstrap selections (either DA or RT)
        sites_with_bootstrap = []
        bootstrap_info = {}  # Track which bootstrap types are available per site
        
        for site in self.available_sites:
            has_da = (self.bootstrap_path_da / f"{site}_bootstrap_selections_da_latest.json").exists()
            has_rt = (self.bootstrap_path_rt / f"{site}_bootstrap_selections_rt_latest.json").exists()
            
            if has_da or has_rt:
                sites_with_bootstrap.append(site)
                bootstrap_info[site] = {'da': has_da, 'rt': has_rt}
        
        if not sites_with_bootstrap:
            print("❌ No bootstrap selections found! Please run Phase 0 (either DA or RT) first.")
            return None
        
        print(f"\n✅ Found bootstrap selections for {len(sites_with_bootstrap)} sites")
        print("\nAvailable sites:")
        for site in sites_with_bootstrap:
            available_types = []
            if bootstrap_info[site]['da']:
                available_types.append('DA')
            if bootstrap_info[site]['rt']:
                available_types.append('RT')
            print(f"   • {site} (bootstrap available: {', '.join(available_types)})")
        
        print("\nOptions:")
        print("0. ALL SITES (Process all sites with bootstrap selections)")
        for i, site in enumerate(sites_with_bootstrap):
            print(f"{i+1}. {site}")
        
        print("="*60)
        
        while True:
            try:
                selection = input("\n📊 Select option number (0 for all sites): ").strip()
                if selection == '0':
                    return 'ALL_SITES', sites_with_bootstrap, bootstrap_info
                else:
                    idx = int(selection) - 1
                    if 0 <= idx < len(sites_with_bootstrap):
                        selected_site = sites_with_bootstrap[idx]
                        return selected_site, [selected_site], bootstrap_info
                    else:
                        print("❌ Invalid selection!")
            except:
                print("❌ Please enter a valid number!")
    
    def load_bootstrap_selections(self, site_name, bootstrap_info):
        """
        Load bootstrap selections for a specific site
        Automatically detects which type to use (prefers DA if both exist)
        """
        # Determine which bootstrap to use
        has_da = bootstrap_info[site_name]['da']
        has_rt = bootstrap_info[site_name]['rt']
        
        if has_da:
            # Prefer DA if available
            bootstrap_file = self.bootstrap_path_da / f"{site_name}_bootstrap_selections_da_latest.json"
            self.bootstrap_type = 'da'
            self.pipeline_name = "DAY-AHEAD"
            self.pipeline_name_short = "DA"
        elif has_rt:
            # Use RT if DA not available
            bootstrap_file = self.bootstrap_path_rt / f"{site_name}_bootstrap_selections_rt_latest.json"
            self.bootstrap_type = 'rt'
            self.pipeline_name = "REAL-TIME"
            self.pipeline_name_short = "RT"
        else:
            print(f"❌ No bootstrap selections found for {site_name}")
            return False
        
        print(f"\n📁 Loading bootstrap selections from: {bootstrap_file.name}")
        print(f"   Using {self.pipeline_name} bootstrap for generation")
        
        with open(bootstrap_file, 'r') as f:
            data = json.load(f)
        
        self.bootstrap_selections = data['selections']
        self.n_synthetic_paths = data['n_paths']
        self.bootstrap_metadata = data['metadata']
        
        print(f"   ✓ Loaded {self.n_synthetic_paths} path selections")
        print(f"   ✓ Years used: {self.bootstrap_metadata['year_range']} ({self.bootstrap_metadata['n_years']} years)")
        print(f"   ✓ Bootstrap type: {self.pipeline_name}")
        
        return True
    
    def get_automatic_month_range(self):
        """
        Automatically determine month range: current month to 11 months later
        """
        current_date = datetime.now()
        current_month = current_date.month
        
        start_month = current_month
        if start_month == 1:
            end_month = 12
        else:
            end_month = start_month - 1
        
        print(f"\n📅 Auto-detected period: {self.month_names[start_month]} to {self.month_names[end_month]} (12 months)")
        print(f"   Starting from current month: {self.month_names[current_month]}")
        
        return start_month, end_month
    
    def get_months_in_range(self, start_month, end_month):
        """
        Get list of months in range, handling year-wrapping
        """
        if start_month <= end_month:
            return list(range(start_month, end_month + 1))
        else:
            return list(range(start_month, 13)) + list(range(1, end_month + 1))
    
    def filter_data_for_months(self, df, start_month, end_month):
        """
        Filter dataframe for month range, handling year-wrapping
        """
        months_in_range = self.get_months_in_range(start_month, end_month)
        return df[df['month'].isin(months_in_range)].copy()
    
    def create_month_order_map(self, start_month, end_month):
        """
        Create a mapping for sorting months in the specified order
        """
        months_in_range = self.get_months_in_range(start_month, end_month)
        return {month: idx for idx, month in enumerate(months_in_range)}
    
    def generate_synthetic_paths_from_bootstrap(self, df):
        """
        Generate synthetic paths using bootstrap selections
        """
        print(f"\n🎲 Generating {self.n_synthetic_paths} synthetic paths from bootstrap selections...")
        
        synthetic_data = {}
        
        for path_name, path_selection in self.bootstrap_selections.items():
            print(f"   Creating {path_name}...", end='')
            path_data = []
            
            # For each month in the selection
            for month_name, selected_year in path_selection.items():
                if selected_year is None:
                    print(f"\n   ⚠️  Warning: No selection for {month_name} in {path_name}")
                    continue
                
                # Get month number
                month_num = self.month_names.index(month_name)
                
                # Get all data for this year-month combination
                month_data = df[(df['year'] == selected_year) & (df['month'] == month_num)]
                
                if len(month_data) > 0:
                    path_data.append(month_data.copy())
                else:
                    print(f"\n   ⚠️  Warning: No data found for {month_name} {selected_year}")
            
            # Combine all months for this path
            if path_data:
                path_df = pd.concat(path_data, ignore_index=True)
                synthetic_data[path_name] = path_df
                print(f" ✓")
            else:
                print(f" ✗ (No data)")
        
        print(f"\n   ✓ Successfully created {len(synthetic_data)} synthetic paths")
        
        return synthetic_data
    
    def create_hourly_timeseries_with_paths(self, df_filtered, synthetic_paths, month_order_map):
        """
        Create hourly timeseries including both historical years and synthetic paths
        """
        print(f"\n⏰ Creating HOURLY timeseries with synthetic paths...")
        
        df_work = df_filtered.copy()
        df_work['month_order'] = df_work['month'].map(month_order_map)
        
        # Create pivot for historical data
        pivot_df = df_work.pivot_table(
            index=['month', 'day', 'hour', 'month_order'],
            columns='year',
            values='generation_mw',
            aggfunc='mean'
        ).reset_index()
        
        # Add synthetic paths
        for path_name, path_data in synthetic_paths.items():
            # Filter path data for the month range
            path_filtered = self.filter_data_for_months(path_data, 
                                                      min(month_order_map.keys()), 
                                                      max(month_order_map.keys()))
            
            # Aggregate by month-day-hour
            path_grouped = path_filtered.groupby(['month', 'day', 'hour'])['generation_mw'].mean()
            
            # Add to pivot dataframe
            pivot_df[path_name] = pivot_df.apply(
                lambda row: path_grouped.get((int(row['month']), int(row['day']), int(row['hour'])), np.nan),
                axis=1
            )
        
        # Sort and format
        pivot_df = pivot_df.sort_values(['month_order', 'day', 'hour']).reset_index(drop=True)
        pivot_df['datetime_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d} {int(row['hour']):02d}:00",
            axis=1
        )
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        # Reorder columns: metadata, historical years, then paths
        year_cols = sorted([col for col in pivot_df.columns if isinstance(col, int)])
        path_cols = sorted([col for col in pivot_df.columns if isinstance(col, str) and col.startswith('path_')])
        cols = ['datetime_label', 'month', 'day', 'hour'] + year_cols + path_cols
        pivot_df = pivot_df[cols]
        
        # Only include years that were part of the overlap
        overlap_years = set(self.bootstrap_metadata['years_used'])
        year_cols_filtered = [col for col in year_cols if col in overlap_years]
        
        print(f"   ✓ Created timeseries with {len(year_cols_filtered)} historical years (overlap) and {len(path_cols)} synthetic paths")
        
        return pivot_df
    
    def create_daily_timeseries_with_paths(self, df_filtered, synthetic_paths, month_order_map):
        """
        Create daily timeseries including both historical years and synthetic paths
        """
        print(f"\n📅 Creating DAILY timeseries with synthetic paths...")
        
        # Aggregate to daily
        df_daily = df_filtered.groupby(['year', 'month', 'day'])['generation_mw'].sum().reset_index()
        df_daily.rename(columns={'generation_mw': 'daily_generation_mwh'}, inplace=True)
        df_daily['month_order'] = df_daily['month'].map(month_order_map)
        
        # Create pivot for historical data
        pivot_df = df_daily.pivot_table(
            index=['month', 'day', 'month_order'],
            columns='year',
            values='daily_generation_mwh',
            aggfunc='sum'
        ).reset_index()
        
        # Add synthetic paths
        for path_name, path_data in synthetic_paths.items():
            # Filter path data for the month range
            path_filtered = self.filter_data_for_months(path_data, 
                                                      min(month_order_map.keys()), 
                                                      max(month_order_map.keys()))
            
            # Aggregate to daily
            path_daily = path_filtered.groupby(['month', 'day'])['generation_mw'].sum()
            
            # Add to pivot dataframe
            pivot_df[path_name] = pivot_df.apply(
                lambda row: path_daily.get((int(row['month']), int(row['day'])), np.nan),
                axis=1
            )
        
        # Sort and format
        pivot_df = pivot_df.sort_values(['month_order', 'day']).reset_index(drop=True)
        pivot_df['date_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d}",
            axis=1
        )
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        # Reorder columns
        year_cols = sorted([col for col in pivot_df.columns if isinstance(col, int)])
        path_cols = sorted([col for col in pivot_df.columns if isinstance(col, str) and col.startswith('path_')])
        cols = ['date_label', 'month', 'day'] + year_cols + path_cols
        pivot_df = pivot_df[cols]
        
        # Only include years that were part of the overlap
        overlap_years = set(self.bootstrap_metadata['years_used'])
        year_cols_filtered = [col for col in year_cols if col in overlap_years]
        
        print(f"   ✓ Created timeseries with {len(year_cols_filtered)} historical years (overlap) and {len(path_cols)} synthetic paths")
        
        return pivot_df
    
    def create_monthly_timeseries_with_paths(self, df_filtered, synthetic_paths, month_order_map):
        """
        Create monthly timeseries including both historical years and synthetic paths
        """
        print(f"\n📊 Creating MONTHLY timeseries with synthetic paths...")
        
        # Aggregate to monthly
        df_monthly = df_filtered.groupby(['year', 'month'])['generation_mw'].sum().reset_index()
        df_monthly.rename(columns={'generation_mw': 'monthly_generation_mwh'}, inplace=True)
        df_monthly['month_order'] = df_monthly['month'].map(month_order_map)
        
        # Create pivot for historical data
        pivot_df = df_monthly.pivot_table(
            index=['month', 'month_order'],
            columns='year',
            values='monthly_generation_mwh',
            aggfunc='sum'
        ).reset_index()
        
        # Add synthetic paths
        for path_name, path_data in synthetic_paths.items():
            # Filter path data for the month range
            path_filtered = self.filter_data_for_months(path_data, 
                                                      min(month_order_map.keys()), 
                                                      max(month_order_map.keys()))
            
            # Aggregate to monthly
            path_monthly = path_filtered.groupby('month')['generation_mw'].sum()
            
            # Add to pivot dataframe
            pivot_df[path_name] = pivot_df['month'].apply(
                lambda month: path_monthly.get(int(month), np.nan)
            )
        
        # Sort and format
        pivot_df = pivot_df.sort_values(['month_order']).reset_index(drop=True)
        pivot_df['month_name'] = pivot_df['month'].apply(lambda x: self.month_names_full[int(x)])
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        # Reorder columns
        year_cols = sorted([col for col in pivot_df.columns if isinstance(col, int)])
        path_cols = sorted([col for col in pivot_df.columns if isinstance(col, str) and col.startswith('path_')])
        cols = ['month_name', 'month'] + year_cols + path_cols
        pivot_df = pivot_df[cols]
        
        # Only include years that were part of the overlap
        overlap_years = set(self.bootstrap_metadata['years_used'])
        year_cols_filtered = [col for col in year_cols if col in overlap_years]
        
        print(f"   ✓ Created timeseries with {len(year_cols_filtered)} historical years (overlap) and {len(path_cols)} synthetic paths")
        
        return pivot_df
    
    def create_historical_continuous_data(self, df, start_month, end_month):
        """
        Create historical data in continuous (long) format WITH WEATHER DATA
        """
        print("\n📜 Creating historical continuous data (with weather)...")
        
        # Filter for month range
        df_filtered = self.filter_data_for_months(df, start_month, end_month)
        
        # Identify which weather columns are present in the data
        all_weather_cols = self.weather_cols_avg + self.weather_cols_sum + self.weather_cols_skip
        available_weather_cols = [col for col in all_weather_cols if col in df_filtered.columns]
        
        if available_weather_cols:
            print(f"   ✓ Found {len(available_weather_cols)} weather variables in data")
        else:
            print(f"   ⚠️  No weather variables found in data")
        
        # HOURLY HISTORICAL - include all weather columns as-is
        hourly_cols = ['datetime', 'year', 'month', 'day', 'hour', 'generation_mw']
        hourly_cols.extend(available_weather_cols)  # Add all available weather columns
        
        hourly_hist = df_filtered[hourly_cols].copy()
        hourly_hist = hourly_hist.sort_values('datetime').reset_index(drop=True)
        
        # DAILY HISTORICAL - aggregate weather appropriately
        daily_agg_dict = {
            'generation_mw': 'sum',
            'datetime': 'first'
        }
        
        # Add weather aggregations
        for col in available_weather_cols:
            if col in self.weather_cols_avg:
                daily_agg_dict[col] = 'mean'
            elif col in self.weather_cols_sum:
                daily_agg_dict[col] = 'sum'
            # Skip wind direction columns for daily aggregation
        
        daily_hist = df_filtered.groupby(['year', 'month', 'day']).agg(daily_agg_dict).reset_index()
        daily_hist['date'] = daily_hist['datetime'].dt.date
        
        # Reorder columns
        daily_cols = ['date', 'year', 'month', 'day', 'generation_mw']
        daily_cols.extend([col for col in available_weather_cols if col not in self.weather_cols_skip])
        daily_hist = daily_hist[daily_cols]
        daily_hist.rename(columns={'generation_mw': 'daily_generation_mwh'}, inplace=True)
        daily_hist = daily_hist.sort_values('date').reset_index(drop=True)
        
        # MONTHLY HISTORICAL - aggregate weather appropriately
        monthly_agg_dict = {'generation_mw': 'sum'}
        
        # Add weather aggregations
        for col in available_weather_cols:
            if col in self.weather_cols_avg:
                monthly_agg_dict[col] = 'mean'
            elif col in self.weather_cols_sum:
                monthly_agg_dict[col] = 'sum'
            # Skip wind direction columns for monthly aggregation
        
        monthly_hist = df_filtered.groupby(['year', 'month']).agg(monthly_agg_dict).reset_index()
        monthly_hist['month_year'] = monthly_hist.apply(
            lambda row: f"{row['year']}-{int(row['month']):02d}", axis=1
        )
        monthly_hist.rename(columns={'generation_mw': 'monthly_generation_mwh'}, inplace=True)
        
        # Reorder columns
        monthly_cols = ['month_year', 'year', 'month', 'monthly_generation_mwh']
        monthly_cols.extend([col for col in available_weather_cols if col not in self.weather_cols_skip])
        monthly_hist = monthly_hist[monthly_cols]
        monthly_hist = monthly_hist.sort_values(['year', 'month']).reset_index(drop=True)
        
        print(f"   ✓ Created historical data:")
        print(f"      - Hourly: {len(hourly_hist)} records (with {len([c for c in available_weather_cols if c in hourly_hist.columns])} weather variables)")
        print(f"      - Daily: {len(daily_hist)} records (with {len([c for c in available_weather_cols if c in daily_hist.columns and c not in self.weather_cols_skip])} weather variables)")
        print(f"      - Monthly: {len(monthly_hist)} records (with {len([c for c in available_weather_cols if c in monthly_hist.columns and c not in self.weather_cols_skip])} weather variables)")
        
        if available_weather_cols:
            print(f"   ℹ️  Weather aggregation:")
            print(f"      - Averaged: {', '.join([c for c in self.weather_cols_avg if c in available_weather_cols])}")
            if any(c in available_weather_cols for c in self.weather_cols_sum):
                print(f"      - Summed: {', '.join([c for c in self.weather_cols_sum if c in available_weather_cols])}")
            if any(c in available_weather_cols for c in self.weather_cols_skip):
                print(f"      - Skipped in daily/monthly: {', '.join([c for c in self.weather_cols_skip if c in available_weather_cols])}")
        
        return hourly_hist, daily_hist, monthly_hist
    
    def save_all_results(self, hourly_ts, daily_ts, monthly_ts,
                        hourly_hist, daily_hist, monthly_hist, site_name):
        """
        Save timeseries and historical files in Generation folder
        NOTE: SHARED folder for both DA and RT pipelines
        """
        # Create folder structure with Generation subfolder
        generation_path = self.base_output_path / site_name / 'Generation'
        forecast_path = generation_path / 'forecast'
        timeseries_path = forecast_path / 'timeseries'
        historical_path = generation_path / 'historical'
        
        timeseries_path.mkdir(parents=True, exist_ok=True)
        historical_path.mkdir(parents=True, exist_ok=True)
        
        # Save timeseries files
        print(f"\n📈 Saving TIMESERIES files (with synthetic paths)...")
        
        # Format and save timeseries files
        for ts_df, ts_name, ts_file in [
            (hourly_ts, "hourly", f"{site_name}_generation_hourly_timeseries.csv"),
            (daily_ts, "daily", f"{site_name}_generation_daily_timeseries.csv"),
            (monthly_ts, "monthly", f"{site_name}_generation_monthly_timeseries.csv")
        ]:
            ts_save = ts_df.copy()
            # Format numeric columns
            numeric_cols = [col for col in ts_save.columns if isinstance(col, int) or (isinstance(col, str) and col.startswith('path_'))]
            for col in numeric_cols:
                ts_save[col] = ts_save[col].apply(lambda x: '' if pd.isna(x) else f'{x:.3f}')
            ts_save.to_csv(timeseries_path / ts_file, index=False)
            print(f"   ✓ {ts_file}")
        
        # Save historical files
        print("\n📜 Saving HISTORICAL files (continuous format with weather)...")
        
        # Hourly historical
        hourly_hist_file = f"{site_name}_generation_hourly_historical.csv"
        hourly_hist.to_csv(historical_path / hourly_hist_file, index=False, float_format='%.3f')
        print(f"   ✓ {hourly_hist_file}")
        
        # Daily historical
        daily_hist_file = f"{site_name}_generation_daily_historical.csv"
        daily_hist.to_csv(historical_path / daily_hist_file, index=False, float_format='%.3f')
        print(f"   ✓ {daily_hist_file}")
        
        # Monthly historical
        monthly_hist_file = f"{site_name}_generation_monthly_historical.csv"
        monthly_hist.to_csv(historical_path / monthly_hist_file, index=False, float_format='%.3f')
        print(f"   ✓ {monthly_hist_file}")
        
        print(f"\n📁 All files saved in:")
        print(f"   • Renewable Portfolio LLC/{site_name}/Generation/forecast/timeseries/")
        print(f"   • Renewable Portfolio LLC/{site_name}/Generation/historical/")
        print(f"\n📌 Note: Generation data is SHARED between DA and RT pipelines")
        print(f"   This run used {self.pipeline_name} bootstrap selections")
        print(f"   Historical files include weather data where available")
        
        # Save metadata about which bootstrap was used
        metadata_file = generation_path / 'generation_metadata.json'
        metadata = {
            'last_updated': datetime.now().isoformat(),
            'bootstrap_type': self.bootstrap_type,
            'pipeline_name': self.pipeline_name,
            'n_synthetic_paths': self.n_synthetic_paths,
            'years_used': self.bootstrap_metadata['years_used'],
            'year_range': self.bootstrap_metadata['year_range'],
            'note': 'Generation data is shared between DA and RT pipelines'
        }
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)
        print(f"   ✓ Saved metadata: generation_metadata.json")
    
    def process_single_site(self, site_name, start_month, end_month, bootstrap_info):
        """
        Process a single site with given month range
        """
        # Load bootstrap selections for this site
        if not self.load_bootstrap_selections(site_name, bootstrap_info):
            return False
        
        # Create month order mapping
        month_order_map = self.create_month_order_map(start_month, end_month)
        
        # Load and prepare data
        print(f"\n{'='*60}")
        print(f"Processing: {site_name}")
        print(f"Using {self.pipeline_name} bootstrap for generation data")
        
        months_in_range = self.get_months_in_range(start_month, end_month)
        num_months = len(months_in_range)
        
        if start_month <= end_month:
            print(f"Month range: {self.month_names[start_month]} to {self.month_names[end_month]} ({num_months} months)")
        else:
            print(f"Month range: {self.month_names[start_month]} to {self.month_names[end_month]} (year-wrapping, {num_months} months)")
        
        print(f"{'='*60}")
        
        # Load combined data
        file_path = self.data_path / f"{site_name}_generation_price_combined.csv"
        print(f"\n📁 Loading data from: {file_path}")
        
        try:
            df = pd.read_csv(file_path)
            df['datetime'] = pd.to_datetime(df['datetime'])
            
            # Ensure numeric columns are integers
            df['year'] = df['year'].astype(int)
            df['month'] = df['month'].astype(int)
            df['hour'] = df['hour'].astype(int)
            df['day'] = df['datetime'].dt.day.astype(int)
            
            # Get full dataset (before filtering) for synthetic path generation
            df_full = df.copy()
            
            # Filter for selected months
            df_filtered = self.filter_data_for_months(df, start_month, end_month)
            
            # Data summary
            years_available = sorted(df_filtered['year'].unique())
            overlap_years = set(self.bootstrap_metadata['years_used'])
            years_in_overlap = sorted([y for y in years_available if y in overlap_years])
            
            print(f"\n📊 Data summary:")
            print(f"   All years available: {years_available[0]} to {years_available[-1]} ({len(years_available)} years)")
            print(f"   Bootstrap overlap years: {years_in_overlap[0]} to {years_in_overlap[-1]} ({len(years_in_overlap)} years)")
            print(f"   Total data points: {len(df_filtered):,}")
            print(f"   Months included: {', '.join([self.month_names[m] for m in months_in_range])}")
            
            # Generate synthetic paths using bootstrap selections
            print("\n" + "-"*40)
            print(f"SYNTHETIC PATH GENERATION FROM BOOTSTRAP")
            print("-"*40)
            synthetic_paths = self.generate_synthetic_paths_from_bootstrap(df_full)
            
            # Create timeseries with synthetic paths
            print("\n" + "-"*40)
            print(f"GENERATION TIMESERIES (HISTORICAL + SYNTHETIC)")
            print("-"*40)
            hourly_ts = self.create_hourly_timeseries_with_paths(df_filtered, synthetic_paths, month_order_map)
            daily_ts = self.create_daily_timeseries_with_paths(df_filtered, synthetic_paths, month_order_map)
            monthly_ts = self.create_monthly_timeseries_with_paths(df_filtered, synthetic_paths, month_order_map)
            
            # Create historical continuous data
            print("\n" + "-"*40)
            print("HISTORICAL DATA PREPARATION")
            print("-"*40)
            hourly_hist, daily_hist, monthly_hist = self.create_historical_continuous_data(df, start_month, end_month)
            
            # Save all results
            print("\n" + "-"*40)
            print("SAVING RESULTS")
            print("-"*40)
            self.save_all_results(hourly_ts, daily_ts, monthly_ts,
                                hourly_hist, daily_hist, monthly_hist,
                                site_name)
            
            return True
            
        except Exception as e:
            print(f"\n❌ Error processing {site_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            return False
    
    def run_preparation(self):
        """
        Main function to run the generation data preparation
        """
        print("\n🌟 Generation Data Preparation - Phase 1a")
        print("   (Shared for both DA and RT pipelines)")
        print("="*60)
        
        # Get site selection
        site_selection_result = self.get_site_selection()
        if not site_selection_result:
            return
        
        site_selection, sites_to_process, bootstrap_info = site_selection_result
        
        # Get automatic month range
        start_month, end_month = self.get_automatic_month_range()
        
        # Process based on selection
        if site_selection == 'ALL_SITES':
            # Process all sites
            print("\n" + "="*60)
            print("🚀 PROCESSING ALL SITES WITH BOOTSTRAP SELECTIONS")
            print("="*60)
            
            successful = 0
            failed = 0
            
            for i, site_name in enumerate(sites_to_process, 1):
                print(f"\n[{i}/{len(sites_to_process)}] Processing {site_name}...")
                
                if self.process_single_site(site_name, start_month, end_month, bootstrap_info):
                    successful += 1
                else:
                    failed += 1
            
            # Summary
            print("\n" + "="*60)
            print("✨ PHASE 1a COMPLETE!")
            print("="*60)
            print(f"\n📊 Summary:")
            print(f"   ✅ Successfully processed: {successful} sites")
            if failed > 0:
                print(f"   ❌ Failed: {failed} sites")
            
            print(f"\n📁 Files saved:")
            print(f"   • Generation/forecast/timeseries/ - Historical years + synthetic paths")
            print(f"   • Generation/historical/ - Continuous format data WITH WEATHER")
            print(f"\n📌 Generation data is SHARED between DA and RT pipelines")
            print(f"   Next step: Run Phase 1b for price data preparation (DA and/or RT)")
            
        else:
            # Process single site
            if self.process_single_site(site_selection, start_month, end_month, bootstrap_info):
                print("\n" + "="*60)
                print("✨ PHASE 1a COMPLETE!")
                print("="*60)
                print(f"\nData prepared for {site_selection}")
                print(f"12-month period: {self.month_names[start_month]} to {self.month_names[end_month]}")
                print(f"Bootstrap used: {self.pipeline_name}")
                print(f"Synthetic paths: {self.n_synthetic_paths}")
                
                print(f"\n📁 Files saved in:")
                print(f"   • Renewable Portfolio LLC/{site_selection}/Generation/forecast/timeseries/")
                print(f"   • Renewable Portfolio LLC/{site_selection}/Generation/historical/ (with weather)")
                
                print(f"\n📌 Generation data is SHARED between DA and RT pipelines")
                print(f"   Next step: Run Phase 1b for price data preparation (DA and/or RT)")
        
        # Ask if user wants to prepare another site
        another = input("\n🔄 Prepare generation data for another site? (y/n): ").strip().lower()
        if another == 'y':
            self.run_preparation()

# Example usage
if __name__ == "__main__":
    print("\n🌟 Generation Data Preparation")
    print("This will create SHARED generation data for both DA and RT pipelines")
    print("The system will automatically use whichever bootstrap is available")
    print("(DA bootstrap is preferred if both exist)")
    
    prep = GenerationDataPreparation()
    prep.run_preparation()


🌟 Generation Data Preparation
This will create SHARED generation data for both DA and RT pipelines
The system will automatically use whichever bootstrap is available
(DA bootstrap is preferred if both exist)

🌟 Generation Data Preparation - Phase 1a
   (Shared for both DA and RT pipelines)

GENERATION DATA PREPARATION - SITE SELECTION
(Shared for both DA and RT pipelines)

✅ Found bootstrap selections for 7 sites

Available sites:
   • Albemarle_Beach_Solar (bootstrap available: DA, RT)
   • Blue_Wing_Solar_Energy_Generator (bootstrap available: DA, RT)
   • Lamesa_Solar (bootstrap available: DA, RT)
   • Midway_Solar_Farm_III (bootstrap available: DA, RT)
   • Misae_Solar (bootstrap available: DA, RT)
   • Mount_Signal_Solar_Farm_II (bootstrap available: DA, RT)
   • RE_Mustang_LLC (bootstrap available: DA, RT)

Options:
0. ALL SITES (Process all sites with bootstrap selections)
1. Albemarle_Beach_Solar
2. Blue_Wing_Solar_Energy_Generator
3. Lamesa_Solar
4. Midway_Solar_Farm_III
5. M

# Phase 1b: Unified price preparation (handles both DA and RT)

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

class PriceDataPreparation:
    """
    Phase 1b: Create price historical data and timeseries (with synthetic paths)
    Unified implementation for both DAY-AHEAD and REAL-TIME pipelines
    Uses bootstrap selections from Phase 0
    Applies compression for forecast files, keeps original for historical
    """
    
    def __init__(self, pipeline_type='da'):
        """
        Initialize the price data preparation
        
        Args:
            pipeline_type (str): 'da' for day-ahead or 'rt' for real-time
        """
        # Validate pipeline type
        if pipeline_type not in ['da', 'rt']:
            raise ValueError("pipeline_type must be 'da' or 'rt'")
        
        self.pipeline_type = pipeline_type
        
        # Define paths
        self.data_path = Path('aamani_data')
        self.base_output_path = Path('Renewable Portfolio LLC')
        
        # Set pipeline-specific configurations
        if pipeline_type == 'da':
            self.bootstrap_path = self.base_output_path / 'bootstrap_selections_da'
            self.price_column = 'price_da'
            self.output_folder = 'Price_da'
            self.file_prefix = 'price_da'
            self.pipeline_name = "DAY-AHEAD"
            self.pipeline_name_short = "DA"
        else:  # rt
            self.bootstrap_path = self.base_output_path / 'bootstrap_selections_rt'
            self.price_column = 'price_rt'  # Default, may be overridden per site
            self.output_folder = 'Price_rt'
            self.file_prefix = 'price_rt'
            self.pipeline_name = "REAL-TIME"
            self.pipeline_name_short = "RT"
        
        # Get available combined files and create mapping
        self.available_files = list(self.data_path.glob('*_generation_price_combined.csv'))
        self.available_sites = []
        self.site_file_map = {}  # Map clean site names to actual filenames
        
        for f in self.available_files:
            # Store the full filename (without path)
            full_filename = f.name
            
            # Extract site name by removing '_generation_price_combined.csv'
            site_name = f.stem.replace('_generation_price_combined', '')
            
            # Clean up the site name by removing '_hourly' to avoid redundancy
            clean_site_name = site_name.replace('_hourly', '')
            
            self.available_sites.append(clean_site_name)
            # Map the clean name to the actual filename
            self.site_file_map[clean_site_name] = full_filename
        
        # Bootstrap selections will be loaded per site
        self.bootstrap_selections = None
        self.n_synthetic_paths = None
        self.bootstrap_metadata = None
        
        # Define compression thresholds (P15 and P85 for compression)
        self.compression_lower = 15
        self.compression_upper = 85
        
        # Month names for labeling
        self.month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        self.month_names_full = ['', 'January', 'February', 'March', 'April', 'May', 'June',
                                'July', 'August', 'September', 'October', 'November', 'December']
    
    def get_site_selection(self):
        """
        Interactive site selection with bootstrap availability check
        """
        print("\n" + "="*60)
        print(f"{self.pipeline_name} PRICE DATA PREPARATION - SITE SELECTION")
        print("="*60)
        
        if not self.available_sites:
            print("❌ No combined generation-price files found in aamani_data!")
            return None
        
        # Check which sites have bootstrap selections
        sites_with_bootstrap = []
        for site in self.available_sites:
            bootstrap_file = self.bootstrap_path / f"{site}_bootstrap_selections_{self.pipeline_type}_latest.json"
            if bootstrap_file.exists():
                # Verify it's actually the correct bootstrap file
                with open(bootstrap_file, 'r') as f:
                    data = json.load(f)
                    expected_type = 'day_ahead' if self.pipeline_type == 'da' else 'real_time'
                    if data.get('pipeline_type') == expected_type:
                        sites_with_bootstrap.append(site)
        
        if not sites_with_bootstrap:
            print(f"❌ No {self.pipeline_name} bootstrap selections found!")
            print(f"   Please run Phase 0 first.")
            return None
        
        print(f"\n✅ Found {self.pipeline_name} bootstrap selections for {len(sites_with_bootstrap)} sites")
        
        print("\nAvailable options:")
        print(f"0. ALL SITES (Process all sites with {self.pipeline_name_short} bootstrap selections)")
        for i, site in enumerate(sites_with_bootstrap):
            print(f"{i+1}. {site}")
        
        print("="*60)
        
        while True:
            try:
                selection = input("\n💰 Select option number (0 for all sites): ").strip()
                if selection == '0':
                    return 'ALL_SITES', sites_with_bootstrap
                else:
                    idx = int(selection) - 1
                    if 0 <= idx < len(sites_with_bootstrap):
                        return sites_with_bootstrap[idx], [sites_with_bootstrap[idx]]
                    else:
                        print("❌ Invalid selection!")
            except:
                print("❌ Please enter a valid number!")
    
    def load_bootstrap_selections(self, site_name):
        """
        Load bootstrap selections for a specific site
        """
        bootstrap_file = self.bootstrap_path / f"{site_name}_bootstrap_selections_{self.pipeline_type}_latest.json"
        
        if not bootstrap_file.exists():
            print(f"❌ {self.pipeline_name} bootstrap selections not found for {site_name}")
            return False
        
        print(f"\n📁 Loading {self.pipeline_name} bootstrap selections from: {bootstrap_file.name}")
        
        with open(bootstrap_file, 'r') as f:
            data = json.load(f)
        
        # Verify this is the correct bootstrap file
        expected_type = 'day_ahead' if self.pipeline_type == 'da' else 'real_time'
        if data.get('pipeline_type') != expected_type:
            print(f"❌ Error: Bootstrap file is not for {self.pipeline_name.lower()} pipeline!")
            return False
        
        self.bootstrap_selections = data['selections']
        self.n_synthetic_paths = data['n_paths']
        self.bootstrap_metadata = data['metadata']
        
        # For RT, get the actual price column used
        if self.pipeline_type == 'rt' and 'price_column' in self.bootstrap_metadata:
            self.price_column = self.bootstrap_metadata['price_column']
        
        print(f"   ✓ Loaded {self.n_synthetic_paths} {self.pipeline_name} path selections")
        print(f"   ✓ Years used: {self.bootstrap_metadata['year_range']} ({self.bootstrap_metadata['n_years']} years)")
        print(f"   ✓ Price type: {self.pipeline_name}")
        if self.pipeline_type == 'rt':
            print(f"   ✓ Price column: {self.price_column}")
        
        return True
    
    def get_automatic_month_range(self):
        """
        Automatically determine month range: current month to 11 months later
        """
        current_date = datetime.now()
        current_month = current_date.month
        
        start_month = current_month
        if start_month == 1:
            end_month = 12
        else:
            end_month = start_month - 1
        
        print(f"\n📅 Auto-detected period: {self.month_names[start_month]} to {self.month_names[end_month]} (12 months)")
        print(f"   Starting from current month: {self.month_names[current_month]}")
        
        return start_month, end_month
    
    def get_months_in_range(self, start_month, end_month):
        """
        Get list of months in range, handling year-wrapping
        """
        if start_month <= end_month:
            return list(range(start_month, end_month + 1))
        else:
            return list(range(start_month, 13)) + list(range(1, end_month + 1))
    
    def filter_data_for_months(self, df, start_month, end_month):
        """
        Filter dataframe for month range, handling year-wrapping
        """
        months_in_range = self.get_months_in_range(start_month, end_month)
        return df[df['month'].isin(months_in_range)].copy()
    
    def create_month_order_map(self, start_month, end_month):
        """
        Create a mapping for sorting months in the specified order
        """
        months_in_range = self.get_months_in_range(start_month, end_month)
        return {month: idx for idx, month in enumerate(months_in_range)}
    
    def compress_outliers(self, values, lower_pct=15, upper_pct=85):
        """
        Compress outliers using logarithmic compression
        """
        # Calculate compression thresholds
        P_lower = np.percentile(values, lower_pct)
        P_upper = np.percentile(values, upper_pct)
        
        # Create compressed values array
        compressed = np.copy(values)
        
        # Compress lower tail (negative extremes)
        lower_mask = values < P_lower
        if np.any(lower_mask):
            offset = P_lower - values[lower_mask]
            compressed[lower_mask] = P_lower - np.log1p(offset)
        
        # Compress upper tail (positive extremes)
        upper_mask = values > P_upper
        if np.any(upper_mask):
            offset = values[upper_mask] - P_upper
            compressed[upper_mask] = P_upper + np.log1p(offset)
        
        return compressed
    
    def calculate_compression_parameters(self, df):
        """
        Calculate P15 and P85 for each month-day-hour slot across all years
        """
        print(f"\n📊 Calculating compression parameters for {self.pipeline_name} prices...")
        
        compression_params = {}
        grouped = df.groupby(['month', 'day', 'hour'])
        
        for (month, day, hour), group in grouped:
            if len(group) >= 5 and group[self.price_column].notna().sum() >= 5:
                prices = group[self.price_column].dropna().values
                P_lower = np.percentile(prices, self.compression_lower)
                P_upper = np.percentile(prices, self.compression_upper)
                compression_params[(month, day, hour)] = (P_lower, P_upper)
        
        print(f"   ✓ Calculated compression parameters for {len(compression_params)} time slots")
        
        return compression_params
    
    def apply_compression_to_dataframe(self, df, compression_params):
        """
        Apply compression to an entire dataframe
        """
        df = df.copy()
        
        def compress_value(row):
            if pd.isna(row[self.price_column]):
                return np.nan
                
            key = (row['month'], row['day'], row['hour'])
            if key in compression_params:
                P_lower, P_upper = compression_params[key]
                value = row[self.price_column]
                
                if value < P_lower:
                    offset = P_lower - value
                    return P_lower - np.log1p(offset)
                elif value > P_upper:
                    offset = value - P_upper
                    return P_upper + np.log1p(offset)
                else:
                    return value
            else:
                return row[self.price_column]
        
        df[f'{self.price_column}_compressed'] = df.apply(compress_value, axis=1)
        
        return df
    
    def generate_synthetic_paths_from_bootstrap(self, df, compression_params):
        """
        Generate synthetic paths using bootstrap selections
        Apply compression to the synthetic paths
        """
        print(f"\n🎲 Generating {self.n_synthetic_paths} synthetic paths from {self.pipeline_name} bootstrap selections...")
        
        synthetic_data = {}
        
        for path_name, path_selection in self.bootstrap_selections.items():
            print(f"   Creating {path_name} ({self.pipeline_name_short})...", end='')
            path_data = []
            
            # For each month in the selection
            for month_name, selected_year in path_selection.items():
                if selected_year is None:
                    continue
                
                # Get month number
                month_num = self.month_names.index(month_name)
                
                # Get all data for this year-month combination
                month_data = df[(df['year'] == selected_year) & (df['month'] == month_num)]
                
                if len(month_data) > 0:
                    # Apply compression to this month's data
                    month_data_compressed = self.apply_compression_to_dataframe(month_data, compression_params)
                    path_data.append(month_data_compressed)
            
            # Combine all months for this path
            if path_data:
                path_df = pd.concat(path_data, ignore_index=True)
                synthetic_data[path_name] = path_df
                print(f" ✓")
            else:
                print(f" ✗ (No data)")
        
        print(f"\n   ✓ Successfully created {len(synthetic_data)} {self.pipeline_name} synthetic paths with compression")
        
        return synthetic_data
    
    def create_hourly_timeseries_with_paths(self, df_filtered, synthetic_paths, month_order_map):
        """
        Create hourly timeseries with COMPRESSED values for both historical and synthetic
        """
        print(f"\n⏰ Creating HOURLY {self.pipeline_name} price timeseries (compressed)...")
        
        df_work = df_filtered.copy()
        df_work['month_order'] = df_work['month'].map(month_order_map)
        
        # Use compressed values for historical data
        price_col_to_use = f'{self.price_column}_compressed'
        
        # Pivot for compressed prices
        pivot_df = df_work.pivot_table(
            index=['month', 'day', 'hour', 'month_order'],
            columns='year',
            values=price_col_to_use,
            aggfunc='mean'
        ).reset_index()
        
        # Add synthetic paths (already compressed)
        for path_name, path_data in synthetic_paths.items():
            path_filtered = self.filter_data_for_months(path_data, 
                                                      min(month_order_map.keys()), 
                                                      max(month_order_map.keys()))
            
            path_grouped = path_filtered.groupby(['month', 'day', 'hour'])[price_col_to_use].mean()
            
            pivot_df[path_name] = pivot_df.apply(
                lambda row: path_grouped.get((int(row['month']), int(row['day']), int(row['hour'])), np.nan),
                axis=1
            )
        
        # Sort and format
        pivot_df = pivot_df.sort_values(['month_order', 'day', 'hour']).reset_index(drop=True)
        pivot_df['datetime_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d} {int(row['hour']):02d}:00",
            axis=1
        )
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        # Reorder columns - only include years that were in overlap
        overlap_years = set(self.bootstrap_metadata['years_used'])
        year_cols = sorted([col for col in pivot_df.columns if isinstance(col, int) and col in overlap_years])
        path_cols = sorted([col for col in pivot_df.columns if isinstance(col, str) and col.startswith('path_')])
        cols = ['datetime_label', 'month', 'day', 'hour'] + year_cols + path_cols
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created compressed timeseries with {len(year_cols)} historical years ({self.pipeline_name_short} overlap) and {len(path_cols)} synthetic paths")
        
        return pivot_df
    
    def create_daily_timeseries_with_paths(self, df_filtered, synthetic_paths, month_order_map):
        """
        Create daily GENERATION-WEIGHTED price timeseries with COMPRESSED values
        """
        print(f"\n📅 Creating DAILY generation-weighted {self.pipeline_name} price timeseries...")
        
        # Calculate weighted daily prices for historical data
        def calc_weighted_price(group):
            mask = group['generation_mw'] > 0
            if mask.sum() == 0:
                return np.nan
            gen_positive = group.loc[mask]
            # Use compressed prices for weighting
            price_col = f'{self.price_column}_compressed'
            return (gen_positive[price_col] * gen_positive['generation_mw']).sum() / gen_positive['generation_mw'].sum()
        
        df_daily = df_filtered.groupby(['year', 'month', 'day']).apply(calc_weighted_price).reset_index()
        df_daily.columns = ['year', 'month', 'day', f'weighted_{self.price_column}']
        df_daily['month_order'] = df_daily['month'].map(month_order_map)
        
        # Create pivot for historical
        pivot_df = df_daily.pivot_table(
            index=['month', 'day', 'month_order'],
            columns='year',
            values=f'weighted_{self.price_column}',
            aggfunc='first'
        ).reset_index()
        
        # Add synthetic paths
        for path_name, path_data in synthetic_paths.items():
            path_filtered = self.filter_data_for_months(path_data, 
                                                      min(month_order_map.keys()), 
                                                      max(month_order_map.keys()))
            
            # Calculate weighted prices for path
            path_daily_weighted = path_filtered.groupby(['month', 'day']).apply(
                lambda g: (g[f'{self.price_column}_compressed'] * g['generation_mw']).sum() / g['generation_mw'].sum() 
                if (g['generation_mw'] > 0).sum() > 0 else np.nan
            )
            
            pivot_df[path_name] = pivot_df.apply(
                lambda row: path_daily_weighted.get((int(row['month']), int(row['day'])), np.nan),
                axis=1
            )
        
        # Sort and format
        pivot_df = pivot_df.sort_values(['month_order', 'day']).reset_index(drop=True)
        pivot_df['date_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d}",
            axis=1
        )
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        # Reorder columns - only include years that were in overlap
        overlap_years = set(self.bootstrap_metadata['years_used'])
        year_cols = sorted([col for col in pivot_df.columns if isinstance(col, int) and col in overlap_years])
        path_cols = sorted([col for col in pivot_df.columns if isinstance(col, str) and col.startswith('path_')])
        cols = ['date_label', 'month', 'day'] + year_cols + path_cols
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created timeseries with {len(year_cols)} historical years ({self.pipeline_name_short} overlap) and {len(path_cols)} synthetic paths")
        
        return pivot_df
    
    def create_monthly_timeseries_with_paths(self, df_filtered, synthetic_paths, month_order_map):
        """
        Create monthly GENERATION-WEIGHTED price timeseries with COMPRESSED values
        """
        print(f"\n📊 Creating MONTHLY generation-weighted {self.pipeline_name} price timeseries...")
        
        # Calculate weighted monthly prices
        def calc_weighted_price(group):
            mask = group['generation_mw'] > 0
            if mask.sum() == 0:
                return np.nan
            gen_positive = group.loc[mask]
            price_col = f'{self.price_column}_compressed'
            return (gen_positive[price_col] * gen_positive['generation_mw']).sum() / gen_positive['generation_mw'].sum()
        
        df_monthly = df_filtered.groupby(['year', 'month']).apply(calc_weighted_price).reset_index()
        df_monthly.columns = ['year', 'month', f'weighted_{self.price_column}']
        df_monthly['month_order'] = df_monthly['month'].map(month_order_map)
        
        # Create pivot
        pivot_df = df_monthly.pivot_table(
            index=['month', 'month_order'],
            columns='year',
            values=f'weighted_{self.price_column}',
            aggfunc='first'
        ).reset_index()
        
        # Add synthetic paths
        for path_name, path_data in synthetic_paths.items():
            path_filtered = self.filter_data_for_months(path_data, 
                                                      min(month_order_map.keys()), 
                                                      max(month_order_map.keys()))
            
            path_monthly_weighted = path_filtered.groupby('month').apply(
                lambda g: (g[f'{self.price_column}_compressed'] * g['generation_mw']).sum() / g['generation_mw'].sum() 
                if (g['generation_mw'] > 0).sum() > 0 else np.nan
            )
            
            pivot_df[path_name] = pivot_df['month'].apply(
                lambda month: path_monthly_weighted.get(int(month), np.nan)
            )
        
        # Sort and format
        pivot_df = pivot_df.sort_values(['month_order']).reset_index(drop=True)
        pivot_df['month_name'] = pivot_df['month'].apply(lambda x: self.month_names_full[int(x)])
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        # Reorder columns - only include years that were in overlap
        overlap_years = set(self.bootstrap_metadata['years_used'])
        year_cols = sorted([col for col in pivot_df.columns if isinstance(col, int) and col in overlap_years])
        path_cols = sorted([col for col in pivot_df.columns if isinstance(col, str) and col.startswith('path_')])
        cols = ['month_name', 'month'] + year_cols + path_cols
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created timeseries with {len(year_cols)} historical years ({self.pipeline_name_short} overlap) and {len(path_cols)} synthetic paths")
        
        return pivot_df
    
    def create_historical_continuous_data(self, df, start_month, end_month):
        """
        Create historical data in continuous format with ORIGINAL (uncompressed) values
        """
        print(f"\n📜 Creating historical continuous data (original {self.pipeline_name} prices)...")
        
        # Filter for month range and valid prices
        df_filtered = self.filter_data_for_months(df, start_month, end_month)
        df_filtered = df_filtered[df_filtered[self.price_column].notna()].copy()
        
        # Hourly historical (original values)
        hourly_hist = df_filtered[['datetime', 'year', 'month', 'day', 'hour', self.price_column, 'generation_mw']].copy()
        hourly_hist = hourly_hist.sort_values('datetime').reset_index(drop=True)
        
        # Daily historical (generation-weighted, original values)
        daily_hist = df_filtered.groupby(['year', 'month', 'day']).apply(
            lambda g: pd.Series({
                'weighted_price': (g[self.price_column] * g['generation_mw']).sum() / g['generation_mw'].sum() 
                                if (g['generation_mw'] > 0).sum() > 0 else np.nan,
                'total_generation': g['generation_mw'].sum(),
                'datetime': g['datetime'].iloc[0]
            })
        ).reset_index()
        daily_hist['date'] = daily_hist['datetime'].dt.date
        daily_hist = daily_hist[['date', 'year', 'month', 'day', 'weighted_price', 'total_generation']]
        daily_hist = daily_hist.sort_values('date').reset_index(drop=True)
        
        # Monthly historical (generation-weighted, original values)
        monthly_hist = df_filtered.groupby(['year', 'month']).apply(
            lambda g: pd.Series({
                'weighted_price': (g[self.price_column] * g['generation_mw']).sum() / g['generation_mw'].sum() 
                                if (g['generation_mw'] > 0).sum() > 0 else np.nan,
                'total_generation': g['generation_mw'].sum()
            })
        ).reset_index()
        monthly_hist['month_year'] = monthly_hist.apply(
            lambda row: f"{row['year']}-{int(row['month']):02d}", axis=1
        )
        monthly_hist = monthly_hist[['month_year', 'year', 'month', 'weighted_price', 'total_generation']]
        monthly_hist = monthly_hist.sort_values(['year', 'month']).reset_index(drop=True)
        
        print(f"   ✓ Created historical data (original {self.pipeline_name} values):")
        print(f"      - Hourly: {len(hourly_hist)} records")
        print(f"      - Daily: {len(daily_hist)} records (generation-weighted)")
        print(f"      - Monthly: {len(monthly_hist)} records (generation-weighted)")
        
        return hourly_hist, daily_hist, monthly_hist
    
    def save_all_results(self, hourly_ts, daily_ts, monthly_ts,
                        hourly_hist, daily_hist, monthly_hist, site_name):
        """
        Save timeseries (compressed) and historical (original) files
        """
        # Create folder structure with proper subfolder
        price_path = self.base_output_path / site_name / self.output_folder
        forecast_path = price_path / 'forecast'
        timeseries_path = forecast_path / 'timeseries'
        historical_path = price_path / 'historical'
        
        timeseries_path.mkdir(parents=True, exist_ok=True)
        historical_path.mkdir(parents=True, exist_ok=True)
        
        # Save timeseries files (COMPRESSED VALUES)
        print(f"\n📈 Saving TIMESERIES files (compressed {self.pipeline_name} prices)...")
        
        # Format and save timeseries files
        for ts_df, ts_name, ts_file in [
            (hourly_ts, "hourly", f"{site_name}_{self.file_prefix}_hourly_timeseries.csv"),
            (daily_ts, "daily", f"{site_name}_{self.file_prefix}_daily_timeseries.csv"),
            (monthly_ts, "monthly", f"{site_name}_{self.file_prefix}_monthly_timeseries.csv")
        ]:
            ts_save = ts_df.copy()
            # Format numeric columns
            numeric_cols = [col for col in ts_save.columns if isinstance(col, int) or (isinstance(col, str) and col.startswith('path_'))]
            for col in numeric_cols:
                ts_save[col] = ts_save[col].apply(lambda x: '' if pd.isna(x) else f'{x:.2f}')
            ts_save.to_csv(timeseries_path / ts_file, index=False)
            print(f"   ✓ {ts_file}")
        
        # Save historical files (ORIGINAL VALUES)
        print(f"\n📜 Saving HISTORICAL files (original {self.pipeline_name} prices)...")
        
        # Hourly historical
        hourly_hist_file = f"{site_name}_{self.file_prefix}_hourly_historical.csv"
        hourly_hist.to_csv(historical_path / hourly_hist_file, index=False, float_format='%.2f')
        print(f"   ✓ {hourly_hist_file}")
        
        # Daily historical (weighted)
        daily_hist_file = f"{site_name}_{self.file_prefix}_daily_historical.csv"
        daily_hist.to_csv(historical_path / daily_hist_file, index=False, float_format='%.2f')
        print(f"   ✓ {daily_hist_file}")
        
        # Monthly historical (weighted)
        monthly_hist_file = f"{site_name}_{self.file_prefix}_monthly_historical.csv"
        monthly_hist.to_csv(historical_path / monthly_hist_file, index=False, float_format='%.2f')
        print(f"   ✓ {monthly_hist_file}")
        
        print(f"\n📁 All files saved in:")
        print(f"   • Renewable Portfolio LLC/{site_name}/{self.output_folder}/forecast/timeseries/ (compressed)")
        print(f"   • Renewable Portfolio LLC/{site_name}/{self.output_folder}/historical/ (original)")
        print(f"\n📌 Note: These files use {self.pipeline_name} bootstrap selections and compressed {self.pipeline_name_short} prices")
    
    def process_single_site(self, site_name, start_month, end_month):
        """
        Process a single site with given month range
        """
        # Load bootstrap selections for this site
        if not self.load_bootstrap_selections(site_name):
            return False
        
        # Create month order mapping
        month_order_map = self.create_month_order_map(start_month, end_month)
        
        # Load and prepare data
        print(f"\n{'='*60}")
        print(f"Processing: {site_name} ({self.pipeline_name} Price Pipeline)")
        
        months_in_range = self.get_months_in_range(start_month, end_month)
        num_months = len(months_in_range)
        
        if start_month <= end_month:
            print(f"Month range: {self.month_names[start_month]} to {self.month_names[end_month]} ({num_months} months)")
        else:
            print(f"Month range: {self.month_names[start_month]} to {self.month_names[end_month]} (year-wrapping, {num_months} months)")
        
        print(f"{'='*60}")
        
        # Use the mapping to get the actual filename
        actual_filename = self.site_file_map.get(site_name)
        if actual_filename:
            file_path = self.data_path / actual_filename
        else:
            # Fallback - try the standard naming convention
            file_path = self.data_path / f"{site_name}_generation_price_combined.csv"
            
        print(f"\n📁 Loading data from: {file_path.name}")
        
        try:
            df = pd.read_csv(file_path)
            df['datetime'] = pd.to_datetime(df['datetime'])
            
            # Ensure numeric columns are integers
            df['year'] = df['year'].astype(int)
            df['month'] = df['month'].astype(int)
            df['hour'] = df['hour'].astype(int)
            df['day'] = df['datetime'].dt.day.astype(int)
            
            # Filter out rows where price is NaN
            print(f"\n🔍 Filtering {self.pipeline_name} price data...")
            total_rows = len(df)
            df_price_valid = df[df[self.price_column].notna()].copy()
            rows_with_price = len(df_price_valid)
            
            print(f"   Total rows: {total_rows:,}")
            print(f"   Rows with valid {self.pipeline_name} prices: {rows_with_price:,}")
            
            # Calculate compression parameters from ALL data (not just filtered months)
            print("\n" + "-"*40)
            print(f"COMPRESSION PARAMETER CALCULATION ({self.pipeline_name})")
            print("-"*40)
            compression_params = self.calculate_compression_parameters(df_price_valid)
            
            # Apply compression to the full dataset
            df_compressed = self.apply_compression_to_dataframe(df_price_valid, compression_params)
            
            # NOW filter for selected months
            df_filtered = self.filter_data_for_months(df_compressed, start_month, end_month)
            
            # Data summary
            years_available = sorted(df_filtered['year'].unique())
            overlap_years = set(self.bootstrap_metadata['years_used'])
            years_in_overlap = sorted([y for y in years_available if y in overlap_years])
            
            print(f"\n📊 Data summary ({self.pipeline_name} price data):")
            print(f"   All years available: {years_available[0]} to {years_available[-1]} ({len(years_available)} years)")
            print(f"   {self.pipeline_name_short} overlap years: {years_in_overlap[0]} to {years_in_overlap[-1]} ({len(years_in_overlap)} years)")
            print(f"   Total data points: {len(df_filtered):,}")
            
            # Generate synthetic paths using bootstrap selections
            print("\n" + "-"*40)
            print(f"SYNTHETIC PATH GENERATION FROM {self.pipeline_name} BOOTSTRAP")
            print("-"*40)
            synthetic_paths = self.generate_synthetic_paths_from_bootstrap(df_compressed, compression_params)
            
            # Create timeseries with synthetic paths (COMPRESSED)
            print("\n" + "-"*40)
            print(f"{self.pipeline_name} PRICE TIMESERIES (COMPRESSED)")
            print("-"*40)
            hourly_ts = self.create_hourly_timeseries_with_paths(df_filtered, synthetic_paths, month_order_map)
            daily_ts = self.create_daily_timeseries_with_paths(df_filtered, synthetic_paths, month_order_map)
            monthly_ts = self.create_monthly_timeseries_with_paths(df_filtered, synthetic_paths, month_order_map)
            
            # Create historical continuous data (ORIGINAL VALUES)
            print("\n" + "-"*40)
            print(f"HISTORICAL DATA PREPARATION (ORIGINAL {self.pipeline_name} VALUES)")
            print("-"*40)
            hourly_hist, daily_hist, monthly_hist = self.create_historical_continuous_data(df, start_month, end_month)
            
            # Save all results
            print("\n" + "-"*40)
            print("SAVING RESULTS")
            print("-"*40)
            self.save_all_results(hourly_ts, daily_ts, monthly_ts,
                                hourly_hist, daily_hist, monthly_hist,
                                site_name)
            
            return True
            
        except Exception as e:
            print(f"\n❌ Error processing {site_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            return False
    
    def run_preparation(self):
        """
        Main function to run the price data preparation
        """
        print(f"\n💰 {self.pipeline_name} Price Data Preparation - Phase 1b")
        print(f"   (Using {self.pipeline_name} Bootstrap Selections)")
        print("="*60)
        
        # Get site selection
        site_selection = self.get_site_selection()
        if not site_selection:
            return
        
        site_selection, sites_to_process = site_selection
        
        # Get automatic month range
        start_month, end_month = self.get_automatic_month_range()
        
        # Process based on selection
        if site_selection == 'ALL_SITES':
            print("\n" + "="*60)
            print(f"🚀 PROCESSING ALL SITES - {self.pipeline_name} Prices")
            print("="*60)
            
            successful = 0
            failed = 0
            
            for i, site_name in enumerate(sites_to_process, 1):
                print(f"\n[{i}/{len(sites_to_process)}] Processing {site_name}...")
                
                if self.process_single_site(site_name, start_month, end_month):
                    successful += 1
                else:
                    failed += 1
            
            # Summary
            print("\n" + "="*60)
            print("✨ PHASE 1b COMPLETE!")
            print("="*60)
            print(f"\n📊 Summary:")
            print(f"   ✅ Successfully processed: {successful} sites")
            if failed > 0:
                print(f"   ❌ Failed: {failed} sites")
            
            print(f"\n📁 Files saved:")
            print(f"   • {self.output_folder}/forecast/timeseries/ - Compressed {self.pipeline_name} prices + synthetic paths")
            print(f"   • {self.output_folder}/historical/ - Original {self.pipeline_name} price values")
            
            if self.pipeline_type == 'da':
                print(f"\n📌 Next step: Run Phase 1c for day-ahead revenue data preparation")
            else:
                print(f"\n📌 Next step: Run Phase 1c for real-time revenue data preparation")
            
        else:
            # Process single site
            if self.process_single_site(site_selection, start_month, end_month):
                print("\n" + "="*60)
                print("✨ PHASE 1b COMPLETE!")
                print("="*60)
                print(f"\n{self.pipeline_name} Price data prepared for {site_selection}")
                print(f"12-month period: {self.month_names[start_month]} to {self.month_names[end_month]}")
                print(f"Synthetic paths: {self.n_synthetic_paths} (from {self.pipeline_name_short} bootstrap selections)")
                
                print(f"\n📁 Files saved in:")
                print(f"   • Renewable Portfolio LLC/{site_selection}/{self.output_folder}/forecast/timeseries/ (compressed)")
                print(f"   • Renewable Portfolio LLC/{site_selection}/{self.output_folder}/historical/ (original)")
                
                print(f"\n💡 Key features:")
                print(f"   • Timeseries use compressed {self.pipeline_name} prices (P15-P85)")
                print(f"   • Historical files preserve original {self.pipeline_name} price values")
                print(f"   • Synthetic paths use same monthly blocks as generation ({self.pipeline_name_short} overlap)")
                print(f"   • Daily/Monthly use generation-weighted prices")
                
                if self.pipeline_type == 'da':
                    print(f"\n📌 Next step: Run Phase 1c for day-ahead revenue calculation")
                else:
                    print(f"\n📌 Next step: Run Phase 1c for real-time revenue calculation")
        
        # Ask if user wants to prepare another site
        another = input(f"\n🔄 Prepare {self.pipeline_name} price data for another site? (y/n): ").strip().lower()
        if another == 'y':
            self.run_preparation()

# Example usage
if __name__ == "__main__":
    # Ask user which pipeline to run
    print("\n💰 Price Data Preparation")
    print("Select pipeline type:")
    print("1. Day-Ahead (DA)")
    print("2. Real-Time (RT)")
    
    while True:
        choice = input("\nEnter your choice (1 or 2): ").strip()
        if choice == '1':
            prep = PriceDataPreparation(pipeline_type='da')
            break
        elif choice == '2':
            prep = PriceDataPreparation(pipeline_type='rt')
            break
        else:
            print("❌ Invalid choice! Please enter 1 or 2.")
    
    prep.run_preparation()


💰 Price Data Preparation
Select pipeline type:
1. Day-Ahead (DA)
2. Real-Time (RT)

💰 REAL-TIME Price Data Preparation - Phase 1b
   (Using REAL-TIME Bootstrap Selections)

REAL-TIME PRICE DATA PREPARATION - SITE SELECTION

✅ Found REAL-TIME bootstrap selections for 7 sites

Available options:
0. ALL SITES (Process all sites with RT bootstrap selections)
1. Albemarle_Beach_Solar
2. Blue_Wing_Solar_Energy_Generator
3. Lamesa_Solar
4. Midway_Solar_Farm_III
5. Misae_Solar
6. Mount_Signal_Solar_Farm_II
7. RE_Mustang_LLC

📅 Auto-detected period: Jul to Jun (12 months)
   Starting from current month: Jul

🚀 PROCESSING ALL SITES - REAL-TIME Prices

[1/7] Processing Albemarle_Beach_Solar...

📁 Loading REAL-TIME bootstrap selections from: Albemarle_Beach_Solar_bootstrap_selections_rt_latest.json
   ✓ Loaded 100 REAL-TIME path selections
   ✓ Years used: 2012-2025 (14 years)
   ✓ Price type: REAL-TIME
   ✓ Price column: price_rt

Processing: Albemarle_Beach_Solar (REAL-TIME Price Pipeline)
Mont

# Phase 1c: Unified revenue preparation (handles both DA and RT)

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

class RevenueDataPreparation:
    """
    Phase 1c: Create revenue historical data and timeseries (with synthetic paths)
    Unified implementation for both DAY-AHEAD and REAL-TIME pipelines
    Reads ORIGINAL data, calculates revenue (Gen × Price), applies compression
    Uses bootstrap selections from Phase 0
    Applies compression for forecast files, keeps original for historical
    """
    
    def __init__(self, pipeline_type='da'):
        """
        Initialize the revenue data preparation
        
        Args:
            pipeline_type (str): 'da' for day-ahead or 'rt' for real-time
        """
        # Validate pipeline type
        if pipeline_type not in ['da', 'rt']:
            raise ValueError("pipeline_type must be 'da' or 'rt'")
        
        self.pipeline_type = pipeline_type
        
        # Define paths
        self.data_path = Path('aamani_data')
        self.base_output_path = Path('Renewable Portfolio LLC')
        
        # Set pipeline-specific configurations
        if pipeline_type == 'da':
            self.bootstrap_path = self.base_output_path / 'bootstrap_selections_da'
            self.price_column = 'price_da'
            self.revenue_column = 'revenue_da'
            self.output_folder = 'Revenue_da'
            self.file_prefix = 'revenue_da'
            self.pipeline_name = "DAY-AHEAD"
            self.pipeline_name_short = "DA"
        else:  # rt
            self.bootstrap_path = self.base_output_path / 'bootstrap_selections_rt'
            self.price_column = 'price_rt'  # Default, may be overridden per site
            self.revenue_column = 'revenue_rt'
            self.output_folder = 'Revenue_rt'
            self.file_prefix = 'revenue_rt'
            self.pipeline_name = "REAL-TIME"
            self.pipeline_name_short = "RT"
        
        # Get available combined files and create mapping
        self.available_files = list(self.data_path.glob('*_generation_price_combined.csv'))
        self.available_sites = []
        self.site_file_map = {}  # Map clean site names to actual filenames
        
        for f in self.available_files:
            # Store the full filename (without path)
            full_filename = f.name
            
            # Extract site name by removing '_generation_price_combined.csv'
            site_name = f.stem.replace('_generation_price_combined', '')
            
            # Clean up the site name by removing '_hourly' to avoid redundancy
            clean_site_name = site_name.replace('_hourly', '')
            
            self.available_sites.append(clean_site_name)
            # Map the clean name to the actual filename
            self.site_file_map[clean_site_name] = full_filename
        
        # Bootstrap selections will be loaded per site
        self.bootstrap_selections = None
        self.n_synthetic_paths = None
        self.bootstrap_metadata = None
        
        # Define compression thresholds (P15 and P85 for compression)
        self.compression_lower = 15
        self.compression_upper = 85
        
        # Month names for labeling
        self.month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        self.month_names_full = ['', 'January', 'February', 'March', 'April', 'May', 'June',
                                'July', 'August', 'September', 'October', 'November', 'December']
    
    def get_site_selection(self):
        """
        Interactive site selection with bootstrap availability check
        """
        print("\n" + "="*60)
        print(f"{self.pipeline_name} REVENUE DATA PREPARATION - SITE SELECTION")
        print("="*60)
        
        if not self.available_sites:
            print("❌ No combined generation-price files found in aamani_data!")
            return None
        
        # Check which sites have bootstrap selections
        sites_with_bootstrap = []
        for site in self.available_sites:
            bootstrap_file = self.bootstrap_path / f"{site}_bootstrap_selections_{self.pipeline_type}_latest.json"
            if bootstrap_file.exists():
                # Verify it's actually the correct bootstrap file
                with open(bootstrap_file, 'r') as f:
                    data = json.load(f)
                    expected_type = 'day_ahead' if self.pipeline_type == 'da' else 'real_time'
                    if data.get('pipeline_type') == expected_type:
                        sites_with_bootstrap.append(site)
        
        if not sites_with_bootstrap:
            print(f"❌ No {self.pipeline_name} bootstrap selections found!")
            print(f"   Please run Phase 0 first.")
            return None
        
        print(f"\n✅ Found {self.pipeline_name} bootstrap selections for {len(sites_with_bootstrap)} sites")
        
        print("\nAvailable options:")
        print(f"0. ALL SITES (Process all sites with {self.pipeline_name_short} bootstrap selections)")
        for i, site in enumerate(sites_with_bootstrap):
            print(f"{i+1}. {site}")
        
        print("="*60)
        
        while True:
            try:
                selection = input("\n💰 Select option number (0 for all sites): ").strip()
                if selection == '0':
                    return 'ALL_SITES', sites_with_bootstrap
                else:
                    idx = int(selection) - 1
                    if 0 <= idx < len(sites_with_bootstrap):
                        return sites_with_bootstrap[idx], [sites_with_bootstrap[idx]]
                    else:
                        print("❌ Invalid selection!")
            except:
                print("❌ Please enter a valid number!")
    
    def load_bootstrap_selections(self, site_name):
        """
        Load bootstrap selections for a specific site
        """
        bootstrap_file = self.bootstrap_path / f"{site_name}_bootstrap_selections_{self.pipeline_type}_latest.json"
        
        if not bootstrap_file.exists():
            print(f"❌ {self.pipeline_name} bootstrap selections not found for {site_name}")
            return False
        
        print(f"\n📁 Loading {self.pipeline_name} bootstrap selections from: {bootstrap_file.name}")
        
        with open(bootstrap_file, 'r') as f:
            data = json.load(f)
        
        # Verify this is the correct bootstrap file
        expected_type = 'day_ahead' if self.pipeline_type == 'da' else 'real_time'
        if data.get('pipeline_type') != expected_type:
            print(f"❌ Error: Bootstrap file is not for {self.pipeline_name.lower()} pipeline!")
            return False
        
        self.bootstrap_selections = data['selections']
        self.n_synthetic_paths = data['n_paths']
        self.bootstrap_metadata = data['metadata']
        
        # For RT, get the actual price column used
        if self.pipeline_type == 'rt' and 'price_column' in self.bootstrap_metadata:
            self.price_column = self.bootstrap_metadata['price_column']
        
        print(f"   ✓ Loaded {self.n_synthetic_paths} {self.pipeline_name} path selections")
        print(f"   ✓ Years used: {self.bootstrap_metadata['year_range']} ({self.bootstrap_metadata['n_years']} years)")
        print(f"   ✓ Price type: {self.pipeline_name}")
        if self.pipeline_type == 'rt':
            print(f"   ✓ Price column: {self.price_column}")
        
        return True
    
    def get_automatic_month_range(self):
        """
        Automatically determine month range: current month to 11 months later
        """
        current_date = datetime.now()
        current_month = current_date.month
        
        start_month = current_month
        if start_month == 1:
            end_month = 12
        else:
            end_month = start_month - 1
        
        print(f"\n📅 Auto-detected period: {self.month_names[start_month]} to {self.month_names[end_month]} (12 months)")
        print(f"   Starting from current month: {self.month_names[current_month]}")
        
        return start_month, end_month
    
    def get_months_in_range(self, start_month, end_month):
        """
        Get list of months in range, handling year-wrapping
        """
        if start_month <= end_month:
            return list(range(start_month, end_month + 1))
        else:
            return list(range(start_month, 13)) + list(range(1, end_month + 1))
    
    def filter_data_for_months(self, df, start_month, end_month):
        """
        Filter dataframe for month range, handling year-wrapping
        """
        months_in_range = self.get_months_in_range(start_month, end_month)
        return df[df['month'].isin(months_in_range)].copy()
    
    def create_month_order_map(self, start_month, end_month):
        """
        Create a mapping for sorting months in the specified order
        """
        months_in_range = self.get_months_in_range(start_month, end_month)
        return {month: idx for idx, month in enumerate(months_in_range)}
    
    def calculate_compression_parameters(self, df):
        """
        Calculate P15 and P85 for each month-day-hour slot across all years for REVENUE
        """
        print(f"\n📊 Calculating compression parameters for {self.pipeline_name} REVENUE...")
        
        compression_params = {}
        grouped = df.groupby(['month', 'day', 'hour'])
        
        for (month, day, hour), group in grouped:
            if len(group) >= 5 and group[self.revenue_column].notna().sum() >= 5:
                revenues = group[self.revenue_column].dropna().values
                P_lower = np.percentile(revenues, self.compression_lower)
                P_upper = np.percentile(revenues, self.compression_upper)
                compression_params[(month, day, hour)] = (P_lower, P_upper)
        
        print(f"   ✓ Calculated compression parameters for {len(compression_params)} time slots")
        
        return compression_params
    
    def apply_compression_to_dataframe(self, df, compression_params):
        """
        Apply compression to revenue values in dataframe
        """
        df = df.copy()
        
        def compress_value(row):
            if pd.isna(row[self.revenue_column]):
                return np.nan
                
            key = (row['month'], row['day'], row['hour'])
            if key in compression_params:
                P_lower, P_upper = compression_params[key]
                value = row[self.revenue_column]
                
                if value < P_lower:
                    offset = P_lower - value
                    return P_lower - np.log1p(offset)
                elif value > P_upper:
                    offset = value - P_upper
                    return P_upper + np.log1p(offset)
                else:
                    return value
            else:
                return row[self.revenue_column]
        
        df[f'{self.revenue_column}_compressed'] = df.apply(compress_value, axis=1)
        
        return df
    
    def generate_synthetic_paths_from_bootstrap(self, df, compression_params):
        """
        Generate synthetic paths using bootstrap selections
        Calculate revenue and apply compression
        """
        print(f"\n🎲 Generating {self.n_synthetic_paths} synthetic revenue paths from {self.pipeline_name} bootstrap...")
        
        synthetic_data = {}
        
        for path_name, path_selection in self.bootstrap_selections.items():
            print(f"   Creating {path_name} ({self.pipeline_name_short} revenue)...", end='')
            path_data = []
            
            # For each month in the selection
            for month_name, selected_year in path_selection.items():
                if selected_year is None:
                    continue
                
                # Get month number
                month_num = self.month_names.index(month_name)
                
                # Get all data for this year-month combination
                month_data = df[(df['year'] == selected_year) & (df['month'] == month_num)].copy()
                
                if len(month_data) > 0:
                    # Calculate revenue for this month
                    month_data[self.revenue_column] = month_data['generation_mw'] * month_data[self.price_column]
                    
                    # Apply compression to this month's revenue
                    month_data_compressed = self.apply_compression_to_dataframe(month_data, compression_params)
                    path_data.append(month_data_compressed)
            
            # Combine all months for this path
            if path_data:
                path_df = pd.concat(path_data, ignore_index=True)
                synthetic_data[path_name] = path_df
                print(f" ✓")
            else:
                print(f" ✗ (No data)")
        
        print(f"\n   ✓ Successfully created {len(synthetic_data)} {self.pipeline_name} revenue paths with compression")
        
        return synthetic_data
    
    def create_hourly_timeseries_with_paths(self, df_filtered, synthetic_paths, month_order_map):
        """
        Create hourly timeseries with COMPRESSED revenue values
        """
        print(f"\n⏰ Creating HOURLY {self.pipeline_name} revenue timeseries (compressed)...")
        
        df_work = df_filtered.copy()
        df_work['month_order'] = df_work['month'].map(month_order_map)
        
        # Use compressed values for historical data
        revenue_col_to_use = f'{self.revenue_column}_compressed'
        
        # Pivot for compressed revenues
        pivot_df = df_work.pivot_table(
            index=['month', 'day', 'hour', 'month_order'],
            columns='year',
            values=revenue_col_to_use,
            aggfunc='mean'
        ).reset_index()
        
        # Add synthetic paths (already compressed)
        for path_name, path_data in synthetic_paths.items():
            path_filtered = self.filter_data_for_months(path_data, 
                                                      min(month_order_map.keys()), 
                                                      max(month_order_map.keys()))
            
            path_grouped = path_filtered.groupby(['month', 'day', 'hour'])[revenue_col_to_use].mean()
            
            pivot_df[path_name] = pivot_df.apply(
                lambda row: path_grouped.get((int(row['month']), int(row['day']), int(row['hour'])), np.nan),
                axis=1
            )
        
        # Sort and format
        pivot_df = pivot_df.sort_values(['month_order', 'day', 'hour']).reset_index(drop=True)
        pivot_df['datetime_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d} {int(row['hour']):02d}:00",
            axis=1
        )
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        # Reorder columns - only include years that were in overlap
        overlap_years = set(self.bootstrap_metadata['years_used'])
        year_cols = sorted([col for col in pivot_df.columns if isinstance(col, int) and col in overlap_years])
        path_cols = sorted([col for col in pivot_df.columns if isinstance(col, str) and col.startswith('path_')])
        cols = ['datetime_label', 'month', 'day', 'hour'] + year_cols + path_cols
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created compressed timeseries with {len(year_cols)} historical years ({self.pipeline_name_short} overlap) and {len(path_cols)} synthetic paths")
        
        return pivot_df
    
    def create_daily_timeseries_with_paths(self, df_filtered, synthetic_paths, month_order_map):
        """
        Create daily revenue timeseries by summing hourly revenues
        """
        print(f"\n📅 Creating DAILY {self.pipeline_name} revenue timeseries...")
        
        # Aggregate hourly revenue to daily
        df_daily = df_filtered.groupby(['year', 'month', 'day'])[f'{self.revenue_column}_compressed'].sum().reset_index()
        df_daily.rename(columns={f'{self.revenue_column}_compressed': f'daily_{self.revenue_column}'}, inplace=True)
        df_daily['month_order'] = df_daily['month'].map(month_order_map)
        
        # Create pivot for historical data
        pivot_df = df_daily.pivot_table(
            index=['month', 'day', 'month_order'],
            columns='year',
            values=f'daily_{self.revenue_column}',
            aggfunc='sum'
        ).reset_index()
        
        # Add synthetic paths
        for path_name, path_data in synthetic_paths.items():
            path_filtered = self.filter_data_for_months(path_data, 
                                                      min(month_order_map.keys()), 
                                                      max(month_order_map.keys()))
            
            # Aggregate to daily
            path_daily = path_filtered.groupby(['month', 'day'])[f'{self.revenue_column}_compressed'].sum()
            
            # Add to pivot dataframe
            pivot_df[path_name] = pivot_df.apply(
                lambda row: path_daily.get((int(row['month']), int(row['day'])), np.nan),
                axis=1
            )
        
        # Sort and format
        pivot_df = pivot_df.sort_values(['month_order', 'day']).reset_index(drop=True)
        pivot_df['date_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d}",
            axis=1
        )
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        # Reorder columns
        overlap_years = set(self.bootstrap_metadata['years_used'])
        year_cols = sorted([col for col in pivot_df.columns if isinstance(col, int) and col in overlap_years])
        path_cols = sorted([col for col in pivot_df.columns if isinstance(col, str) and col.startswith('path_')])
        cols = ['date_label', 'month', 'day'] + year_cols + path_cols
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created timeseries with {len(year_cols)} historical years ({self.pipeline_name_short} overlap) and {len(path_cols)} synthetic paths")
        
        return pivot_df
    
    def create_monthly_timeseries_with_paths(self, df_filtered, synthetic_paths, month_order_map):
        """
        Create monthly revenue timeseries by summing daily revenues
        """
        print(f"\n📊 Creating MONTHLY {self.pipeline_name} revenue timeseries...")
        
        # Aggregate to monthly
        df_monthly = df_filtered.groupby(['year', 'month'])[f'{self.revenue_column}_compressed'].sum().reset_index()
        df_monthly.rename(columns={f'{self.revenue_column}_compressed': f'monthly_{self.revenue_column}'}, inplace=True)
        df_monthly['month_order'] = df_monthly['month'].map(month_order_map)
        
        # Create pivot
        pivot_df = df_monthly.pivot_table(
            index=['month', 'month_order'],
            columns='year',
            values=f'monthly_{self.revenue_column}',
            aggfunc='sum'
        ).reset_index()
        
        # Add synthetic paths
        for path_name, path_data in synthetic_paths.items():
            path_filtered = self.filter_data_for_months(path_data, 
                                                      min(month_order_map.keys()), 
                                                      max(month_order_map.keys()))
            
            path_monthly = path_filtered.groupby('month')[f'{self.revenue_column}_compressed'].sum()
            
            pivot_df[path_name] = pivot_df['month'].apply(
                lambda month: path_monthly.get(int(month), np.nan)
            )
        
        # Sort and format
        pivot_df = pivot_df.sort_values(['month_order']).reset_index(drop=True)
        pivot_df['month_name'] = pivot_df['month'].apply(lambda x: self.month_names_full[int(x)])
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        # Reorder columns
        overlap_years = set(self.bootstrap_metadata['years_used'])
        year_cols = sorted([col for col in pivot_df.columns if isinstance(col, int) and col in overlap_years])
        path_cols = sorted([col for col in pivot_df.columns if isinstance(col, str) and col.startswith('path_')])
        cols = ['month_name', 'month'] + year_cols + path_cols
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created timeseries with {len(year_cols)} historical years ({self.pipeline_name_short} overlap) and {len(path_cols)} synthetic paths")
        
        return pivot_df
    
    def create_historical_continuous_data(self, df, start_month, end_month):
        """
        Create historical data in continuous format with ORIGINAL (uncompressed) revenue values
        """
        print(f"\n📜 Creating historical continuous data (original {self.pipeline_name} revenue)...")
        
        # Filter for month range and valid prices AND generation
        df_filtered = self.filter_data_for_months(df, start_month, end_month)
        df_filtered = df_filtered[(df_filtered[self.price_column].notna()) & 
                                  (df_filtered['generation_mw'].notna())].copy()
        
        # Calculate revenue (original values)
        df_filtered[self.revenue_column] = df_filtered['generation_mw'] * df_filtered[self.price_column]
        
        # Hourly historical (original values)
        hourly_hist = df_filtered[['datetime', 'year', 'month', 'day', 'hour', 
                                   'generation_mw', self.price_column, self.revenue_column]].copy()
        hourly_hist = hourly_hist.sort_values('datetime').reset_index(drop=True)
        
        # Daily historical
        daily_hist = df_filtered.groupby(['year', 'month', 'day']).agg({
            self.revenue_column: 'sum',
            'generation_mw': 'sum',
            'datetime': 'first'
        }).reset_index()
        
        # Calculate weighted average price for daily
        daily_hist[f'weighted_{self.price_column}'] = daily_hist[self.revenue_column] / daily_hist['generation_mw']
        daily_hist['date'] = pd.to_datetime(daily_hist['datetime']).dt.date
        daily_hist = daily_hist[['date', 'year', 'month', 'day', self.revenue_column, 'generation_mw', f'weighted_{self.price_column}']]
        daily_hist = daily_hist.sort_values('date').reset_index(drop=True)
        
        # Monthly historical
        monthly_hist = df_filtered.groupby(['year', 'month']).agg({
            self.revenue_column: 'sum',
            'generation_mw': 'sum'
        }).reset_index()
        
        # Calculate weighted average price for monthly
        monthly_hist[f'weighted_{self.price_column}'] = monthly_hist[self.revenue_column] / monthly_hist['generation_mw']
        monthly_hist['month_year'] = monthly_hist.apply(
            lambda row: f"{row['year']}-{int(row['month']):02d}", axis=1
        )
        monthly_hist = monthly_hist[['month_year', 'year', 'month', self.revenue_column, 'generation_mw', f'weighted_{self.price_column}']]
        monthly_hist = monthly_hist.sort_values(['year', 'month']).reset_index(drop=True)
        
        print(f"   ✓ Created historical data (original {self.pipeline_name} revenue):")
        print(f"      - Hourly: {len(hourly_hist)} records")
        print(f"      - Daily: {len(daily_hist)} records")
        print(f"      - Monthly: {len(monthly_hist)} records")
        
        return hourly_hist, daily_hist, monthly_hist
    
    def save_all_results(self, hourly_ts, daily_ts, monthly_ts,
                        hourly_hist, daily_hist, monthly_hist, site_name):
        """
        Save timeseries (compressed) and historical (original) files
        """
        # Create folder structure with proper subfolder
        revenue_path = self.base_output_path / site_name / self.output_folder
        forecast_path = revenue_path / 'forecast'
        timeseries_path = forecast_path / 'timeseries'
        historical_path = revenue_path / 'historical'
        
        timeseries_path.mkdir(parents=True, exist_ok=True)
        historical_path.mkdir(parents=True, exist_ok=True)
        
        # Save timeseries files (COMPRESSED VALUES)
        print(f"\n📈 Saving TIMESERIES files (compressed {self.pipeline_name} revenue)...")
        
        # Format and save timeseries files
        for ts_df, ts_name, ts_file in [
            (hourly_ts, "hourly", f"{site_name}_{self.file_prefix}_hourly_timeseries.csv"),
            (daily_ts, "daily", f"{site_name}_{self.file_prefix}_daily_timeseries.csv"),
            (monthly_ts, "monthly", f"{site_name}_{self.file_prefix}_monthly_timeseries.csv")
        ]:
            ts_save = ts_df.copy()
            # Format numeric columns
            numeric_cols = [col for col in ts_save.columns if isinstance(col, int) or (isinstance(col, str) and col.startswith('path_'))]
            for col in numeric_cols:
                ts_save[col] = ts_save[col].apply(lambda x: '' if pd.isna(x) else f'{x:.2f}')
            ts_save.to_csv(timeseries_path / ts_file, index=False)
            print(f"   ✓ {ts_file}")
        
        # Save historical files (ORIGINAL VALUES)
        print(f"\n📜 Saving HISTORICAL files (original {self.pipeline_name} revenue)...")
        
        # Hourly historical
        hourly_hist_file = f"{site_name}_{self.file_prefix}_hourly_historical.csv"
        hourly_hist.to_csv(historical_path / hourly_hist_file, index=False, float_format='%.2f')
        print(f"   ✓ {hourly_hist_file}")
        
        # Daily historical
        daily_hist_file = f"{site_name}_{self.file_prefix}_daily_historical.csv"
        daily_hist.to_csv(historical_path / daily_hist_file, index=False, float_format='%.2f')
        print(f"   ✓ {daily_hist_file}")
        
        # Monthly historical
        monthly_hist_file = f"{site_name}_{self.file_prefix}_monthly_historical.csv"
        monthly_hist.to_csv(historical_path / monthly_hist_file, index=False, float_format='%.2f')
        print(f"   ✓ {monthly_hist_file}")
        
        print(f"\n📁 All files saved in:")
        print(f"   • Renewable Portfolio LLC/{site_name}/{self.output_folder}/forecast/timeseries/ (compressed)")
        print(f"   • Renewable Portfolio LLC/{site_name}/{self.output_folder}/historical/ (original)")
        print(f"\n📌 Note: Revenue calculated from original data, then compressed for forecast")
    
    def process_single_site(self, site_name, start_month, end_month):
        """
        Process a single site with given month range
        """
        # Load bootstrap selections for this site
        if not self.load_bootstrap_selections(site_name):
            return False
        
        # Create month order mapping
        month_order_map = self.create_month_order_map(start_month, end_month)
        
        # Load and prepare data
        print(f"\n{'='*60}")
        print(f"Processing: {site_name} ({self.pipeline_name} Revenue Pipeline)")
        
        months_in_range = self.get_months_in_range(start_month, end_month)
        num_months = len(months_in_range)
        
        if start_month <= end_month:
            print(f"Month range: {self.month_names[start_month]} to {self.month_names[end_month]} ({num_months} months)")
        else:
            print(f"Month range: {self.month_names[start_month]} to {self.month_names[end_month]} (year-wrapping, {num_months} months)")
        
        print(f"{'='*60}")
        
        # Use the mapping to get the actual filename
        actual_filename = self.site_file_map.get(site_name)
        if actual_filename:
            file_path = self.data_path / actual_filename
        else:
            # Fallback - try the standard naming convention
            file_path = self.data_path / f"{site_name}_generation_price_combined.csv"
            
        print(f"\n📁 Loading data from: {file_path.name}")
        
        try:
            df = pd.read_csv(file_path)
            df['datetime'] = pd.to_datetime(df['datetime'])
            
            # Ensure numeric columns are integers
            df['year'] = df['year'].astype(int)
            df['month'] = df['month'].astype(int)
            df['hour'] = df['hour'].astype(int)
            df['day'] = df['datetime'].dt.day.astype(int)
            
            # Filter for valid generation AND price data
            print(f"\n🔍 Filtering data with valid generation AND {self.pipeline_name} prices...")
            total_rows = len(df)
            df_valid = df[(df['generation_mw'].notna()) & (df[self.price_column].notna())].copy()
            
            # Calculate revenue
            df_valid[self.revenue_column] = df_valid['generation_mw'] * df_valid[self.price_column]
            rows_with_revenue = len(df_valid)
            
            print(f"   Total rows: {total_rows:,}")
            print(f"   Rows with valid revenue data: {rows_with_revenue:,}")
            
            # Calculate compression parameters from ALL data (not just filtered months)
            print("\n" + "-"*40)
            print(f"COMPRESSION PARAMETER CALCULATION ({self.pipeline_name} REVENUE)")
            print("-"*40)
            compression_params = self.calculate_compression_parameters(df_valid)
            
            # Apply compression to the full dataset
            df_compressed = self.apply_compression_to_dataframe(df_valid, compression_params)
            
            # NOW filter for selected months
            df_filtered = self.filter_data_for_months(df_compressed, start_month, end_month)
            
            # Data summary
            years_available = sorted(df_filtered['year'].unique())
            overlap_years = set(self.bootstrap_metadata['years_used'])
            years_in_overlap = sorted([y for y in years_available if y in overlap_years])
            
            print(f"\n📊 Data summary ({self.pipeline_name} revenue data):")
            print(f"   All years available: {years_available[0]} to {years_available[-1]} ({len(years_available)} years)")
            print(f"   {self.pipeline_name_short} overlap years: {years_in_overlap[0]} to {years_in_overlap[-1]} ({len(years_in_overlap)} years)")
            print(f"   Total data points: {len(df_filtered):,}")
            
            # Generate synthetic paths using bootstrap selections
            print("\n" + "-"*40)
            print(f"SYNTHETIC PATH GENERATION FROM {self.pipeline_name} BOOTSTRAP")
            print("-"*40)
            synthetic_paths = self.generate_synthetic_paths_from_bootstrap(df_valid, compression_params)
            
            # Create timeseries with synthetic paths (COMPRESSED)
            print("\n" + "-"*40)
            print(f"{self.pipeline_name} REVENUE TIMESERIES (COMPRESSED)")
            print("-"*40)
            hourly_ts = self.create_hourly_timeseries_with_paths(df_filtered, synthetic_paths, month_order_map)
            daily_ts = self.create_daily_timeseries_with_paths(df_filtered, synthetic_paths, month_order_map)
            monthly_ts = self.create_monthly_timeseries_with_paths(df_filtered, synthetic_paths, month_order_map)
            
            # Create historical continuous data (ORIGINAL VALUES)
            print("\n" + "-"*40)
            print(f"HISTORICAL DATA PREPARATION (ORIGINAL {self.pipeline_name} REVENUE)")
            print("-"*40)
            hourly_hist, daily_hist, monthly_hist = self.create_historical_continuous_data(df, start_month, end_month)
            
            # Save all results
            print("\n" + "-"*40)
            print("SAVING RESULTS")
            print("-"*40)
            self.save_all_results(hourly_ts, daily_ts, monthly_ts,
                                hourly_hist, daily_hist, monthly_hist,
                                site_name)
            
            return True
            
        except Exception as e:
            print(f"\n❌ Error processing {site_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            return False
    
    def run_preparation(self):
        """
        Main function to run the revenue data preparation
        """
        print(f"\n💰 {self.pipeline_name} Revenue Data Preparation - Phase 1c")
        print(f"   (Calculate revenue from original data and apply compression)")
        print("="*60)
        
        # Get site selection
        site_selection = self.get_site_selection()
        if not site_selection:
            return
        
        site_selection, sites_to_process = site_selection
        
        # Get automatic month range
        start_month, end_month = self.get_automatic_month_range()
        
        # Process based on selection
        if site_selection == 'ALL_SITES':
            print("\n" + "="*60)
            print(f"🚀 PROCESSING ALL SITES - {self.pipeline_name} Revenue")
            print("="*60)
            
            successful = 0
            failed = 0
            
            for i, site_name in enumerate(sites_to_process, 1):
                print(f"\n[{i}/{len(sites_to_process)}] Processing {site_name}...")
                
                if self.process_single_site(site_name, start_month, end_month):
                    successful += 1
                else:
                    failed += 1
            
            # Summary
            print("\n" + "="*60)
            print("✨ PHASE 1c COMPLETE!")
            print("="*60)
            print(f"\n📊 Summary:")
            print(f"   ✅ Successfully processed: {successful} sites")
            if failed > 0:
                print(f"   ❌ Failed: {failed} sites")
            
            print(f"\n📁 Files saved:")
            print(f"   • {self.output_folder}/forecast/timeseries/ - Compressed {self.pipeline_name} revenue + synthetic paths")
            print(f"   • {self.output_folder}/historical/ - Original {self.pipeline_name} revenue values")
            
            print(f"\n📌 Next step: Run Phase 2 for all distribution calculations")
            
        else:
            # Process single site
            if self.process_single_site(site_selection, start_month, end_month):
                print("\n" + "="*60)
                print("✨ PHASE 1c COMPLETE!")
                print("="*60)
                print(f"\n{self.pipeline_name} Revenue data prepared for {site_selection}")
                print(f"12-month period: {self.month_names[start_month]} to {self.month_names[end_month]}")
                print(f"Synthetic paths: {self.n_synthetic_paths} (from {self.pipeline_name_short} bootstrap selections)")
                
                print(f"\n📁 Files saved in:")
                print(f"   • Renewable Portfolio LLC/{site_selection}/{self.output_folder}/forecast/timeseries/ (compressed)")
                print(f"   • Renewable Portfolio LLC/{site_selection}/{self.output_folder}/historical/ (original)")
                
                print(f"\n💡 Key features:")
                print(f"   • Revenue calculated from original Gen × {self.pipeline_name_short} Price data")
                print(f"   • Timeseries use compressed revenue values (P15-P85)")
                print(f"   • Historical files preserve original revenue values")
                print(f"   • Synthetic paths use same monthly blocks as generation")
                
                print(f"\n📌 Next step: Run Phase 2 for all distribution calculations")
        
        # Ask if user wants to prepare another site
        another = input(f"\n🔄 Prepare {self.pipeline_name} revenue data for another site? (y/n): ").strip().lower()
        if another == 'y':
            self.run_preparation()

# Example usage
if __name__ == "__main__":
    # Ask user which pipeline to run
    print("\n💰 Revenue Data Preparation")
    print("Select pipeline type:")
    print("1. Day-Ahead (DA)")
    print("2. Real-Time (RT)")
    
    while True:
        choice = input("\nEnter your choice (1 or 2): ").strip()
        if choice == '1':
            prep = RevenueDataPreparation(pipeline_type='da')
            break
        elif choice == '2':
            prep = RevenueDataPreparation(pipeline_type='rt')
            break
        else:
            print("❌ Invalid choice! Please enter 1 or 2.")
    
    prep.run_preparation()


💰 Revenue Data Preparation
Select pipeline type:
1. Day-Ahead (DA)
2. Real-Time (RT)

💰 REAL-TIME Revenue Data Preparation - Phase 1c
   (Calculate revenue from original data and apply compression)

REAL-TIME REVENUE DATA PREPARATION - SITE SELECTION

✅ Found REAL-TIME bootstrap selections for 7 sites

Available options:
0. ALL SITES (Process all sites with RT bootstrap selections)
1. Albemarle_Beach_Solar
2. Blue_Wing_Solar_Energy_Generator
3. Lamesa_Solar
4. Midway_Solar_Farm_III
5. Misae_Solar
6. Mount_Signal_Solar_Farm_II
7. RE_Mustang_LLC

📅 Auto-detected period: Jul to Jun (12 months)
   Starting from current month: Jul

🚀 PROCESSING ALL SITES - REAL-TIME Revenue

[1/7] Processing Albemarle_Beach_Solar...

📁 Loading REAL-TIME bootstrap selections from: Albemarle_Beach_Solar_bootstrap_selections_rt_latest.json
   ✓ Loaded 100 REAL-TIME path selections
   ✓ Years used: 2012-2025 (14 years)
   ✓ Price type: REAL-TIME
   ✓ Price column: price_rt

Processing: Albemarle_Beach_Solar (R

# Phase 2: Unified distribution calculator

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

class DistributionCalculator:
    """
    Phase 2: Calculate distributions from timeseries files
    Unified implementation for both DAY-AHEAD and REAL-TIME pipelines
    Handles Generation, Price, and Revenue distributions
    Assumes all timeseries files have been created by Phase 1a, 1b, and 1c
    """
    
    def __init__(self, pipeline_type='da'):
        """
        Initialize the distribution calculator
        
        Args:
            pipeline_type (str): 'da' for day-ahead or 'rt' for real-time
        """
        # Validate pipeline type
        if pipeline_type not in ['da', 'rt']:
            raise ValueError("pipeline_type must be 'da' or 'rt'")
        
        self.pipeline_type = pipeline_type
        
        # Define paths
        self.base_path = Path('Renewable Portfolio LLC')
        
        # Set pipeline-specific configurations
        if pipeline_type == 'da':
            self.price_folder = 'Price_da'
            self.revenue_folder = 'Revenue_da'
            self.price_prefix = 'price_da'
            self.revenue_prefix = 'revenue_da'
            self.pipeline_name = "DAY-AHEAD"
            self.pipeline_name_short = "DA"
        else:  # rt
            self.price_folder = 'Price_rt'
            self.revenue_folder = 'Revenue_rt'
            self.price_prefix = 'price_rt'
            self.revenue_prefix = 'revenue_rt'
            self.pipeline_name = "REAL-TIME"
            self.pipeline_name_short = "RT"
        
        # Get available sites
        self.available_sites = []
        if self.base_path.exists():
            for site_dir in self.base_path.iterdir():
                if site_dir.is_dir() and site_dir.name not in ['bootstrap_selections', 'bootstrap_selections_da', 'bootstrap_selections_rt']:
                    self.available_sites.append(site_dir.name)
        
        # Define percentiles to calculate
        self.percentiles = [1, 5, 10, 15, 25, 50, 75, 85, 90, 95, 99]
        
        # Month names for formatting
        self.month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        self.month_names_full = ['', 'January', 'February', 'March', 'April', 'May', 'June',
                                'July', 'August', 'September', 'October', 'November', 'December']
    
    def get_site_selection(self):
        """
        Interactive site selection from available sites
        Checks for all three data types: Generation, Price, and Revenue
        """
        print("\n" + "="*60)
        print(f"{self.pipeline_name} DISTRIBUTION CALCULATOR - SITE SELECTION")
        print("="*60)
        
        if not self.available_sites:
            print("❌ No sites found in Renewable Portfolio LLC!")
            print("   Please run Phase 1 first to generate timeseries files.")
            return None
        
        # Check which sites have complete data for this pipeline
        sites_ready = []
        for site in self.available_sites:
            site_path = self.base_path / site
            
            # Check for all three data types
            gen_ts_path = site_path / 'Generation' / 'forecast' / 'timeseries'
            price_ts_path = site_path / self.price_folder / 'forecast' / 'timeseries'
            revenue_ts_path = site_path / self.revenue_folder / 'forecast' / 'timeseries'
            
            if gen_ts_path.exists() and price_ts_path.exists() and revenue_ts_path.exists():
                sites_ready.append(site)
        
        if not sites_ready:
            print(f"❌ No sites with complete {self.pipeline_name} timeseries data found!")
            print(f"   Please run Phases 1a, 1b, and 1c first.")
            return None
        
        print(f"\n✅ Found {len(sites_ready)} sites with complete {self.pipeline_name} data")
        print("   (Generation, Price, and Revenue timeseries)")
        
        print("\nAvailable options:")
        print("0. ALL SITES (Process all ready sites)")
        for i, site in enumerate(sites_ready):
            print(f"{i+1}. {site}")
        
        print("="*60)
        
        while True:
            try:
                selection = input("\n📊 Select option number (0 for all sites): ").strip()
                if selection == '0':
                    return 'ALL_SITES', sites_ready
                else:
                    idx = int(selection) - 1
                    if 0 <= idx < len(sites_ready):
                        return sites_ready[idx], [sites_ready[idx]]
                    else:
                        print("❌ Invalid selection!")
            except:
                print("❌ Please enter a valid number!")
    
    def get_forecast_year(self, month):
        """
        Determine the forecast year for a given month based on current date
        """
        current_date = datetime.now()
        current_year = current_date.year
        current_month = current_date.month
        
        # Ensure month is an integer
        month = int(month)
        
        if month >= current_month:
            return current_year
        else:
            return current_year + 1
    
    def get_scenario_columns(self, df):
        """
        Get all scenario columns (years and paths) from the dataframe
        """
        scenario_cols = []
        for col in df.columns:
            if isinstance(col, str):
                if col.startswith('path_') or col.isdigit():
                    scenario_cols.append(col)
            elif isinstance(col, int):
                scenario_cols.append(col)
        
        return scenario_cols
    
    def calculate_hourly_distribution(self, hourly_ts_path, data_type='generation'):
        """
        Calculate hourly distribution from timeseries file
        """
        print(f"\n⚡ Calculating HOURLY {data_type} distribution...")
        
        # Read the timeseries file
        df = pd.read_csv(hourly_ts_path)
        
        # Get scenario columns
        scenario_cols = self.get_scenario_columns(df)
        print(f"   Using {len(scenario_cols)} scenarios (historical {self.pipeline_name_short} years + synthetic paths)")
        
        results = []
        
        # For each hour in the timeseries
        for idx, row in df.iterrows():
            # Get values from all scenarios
            values = []
            for col in scenario_cols:
                val = row[col]
                if pd.notna(val) and val != '':
                    try:
                        values.append(float(val))
                    except:
                        pass
            
            if len(values) >= 5:  # Need at least 5 values for statistics
                values_array = np.array(values)
                
                forecast_year = self.get_forecast_year(row['month'])
                
                stats = {
                    'datetime_label': row['datetime_label'],
                    'year': forecast_year,
                    'month': int(row['month']),
                    'day': int(row['day']),
                    'hour': int(row['hour']),
                    'mean': values_array.mean(),
                    'std_dev': values_array.std(),
                    'count': len(values),
                    'min': values_array.min(),
                    'max': values_array.max()
                }
                
                # Calculate percentiles
                for p in self.percentiles:
                    stats[f'p{p}'] = np.percentile(values_array, p)
                
                results.append(stats)
        
        results_df = pd.DataFrame(results)
        print(f"   ✓ Calculated distribution for {len(results_df)} hourly slots")
        
        return results_df
    
    def calculate_daily_distribution(self, daily_ts_path, data_type='generation'):
        """
        Calculate daily distribution from timeseries file
        """
        print(f"\n📅 Calculating DAILY {data_type} distribution...")
        
        # Read the timeseries file
        df = pd.read_csv(daily_ts_path)
        
        # Get scenario columns
        scenario_cols = self.get_scenario_columns(df)
        print(f"   Using {len(scenario_cols)} scenarios (historical {self.pipeline_name_short} years + synthetic paths)")
        
        results = []
        
        # For each day in the timeseries
        for idx, row in df.iterrows():
            # Get values from all scenarios
            values = []
            for col in scenario_cols:
                val = row[col]
                if pd.notna(val) and val != '':
                    try:
                        values.append(float(val))
                    except:
                        pass
            
            if len(values) >= 5:  # Need at least 5 values for statistics
                values_array = np.array(values)
                
                forecast_year = self.get_forecast_year(row['month'])
                
                stats = {
                    'date_label': row['date_label'],
                    'year': forecast_year,
                    'month': int(row['month']),
                    'day': int(row['day']),
                    'mean': values_array.mean(),
                    'std_dev': values_array.std(),
                    'count': len(values),
                    'min': values_array.min(),
                    'max': values_array.max()
                }
                
                # Calculate percentiles
                for p in self.percentiles:
                    stats[f'p{p}'] = np.percentile(values_array, p)
                
                results.append(stats)
        
        results_df = pd.DataFrame(results)
        print(f"   ✓ Calculated distribution for {len(results_df)} daily slots")
        
        return results_df
    
    def calculate_monthly_distribution(self, monthly_ts_path, data_type='generation'):
        """
        Calculate monthly distribution from timeseries file
        """
        print(f"\n📊 Calculating MONTHLY {data_type} distribution...")
        
        # Read the timeseries file
        df = pd.read_csv(monthly_ts_path)
        
        # Get scenario columns
        scenario_cols = self.get_scenario_columns(df)
        print(f"   Using {len(scenario_cols)} scenarios (historical {self.pipeline_name_short} years + synthetic paths)")
        
        results = []
        
        # For each month in the timeseries
        for idx, row in df.iterrows():
            # Get values from all scenarios
            values = []
            for col in scenario_cols:
                val = row[col]
                if pd.notna(val) and val != '':
                    try:
                        values.append(float(val))
                    except:
                        pass
            
            if len(values) >= 5:  # Need at least 5 values for statistics
                values_array = np.array(values)
                
                month_idx = int(row['month'])
                forecast_year = self.get_forecast_year(month_idx)
                
                stats = {
                    'month_name': row['month_name'],
                    'year': forecast_year,
                    'month': month_idx,
                    'mean': values_array.mean(),
                    'std_dev': values_array.std(),
                    'count': len(values),
                    'min': values_array.min(),
                    'max': values_array.max()
                }
                
                # Calculate percentiles
                for p in self.percentiles:
                    stats[f'p{p}'] = np.percentile(values_array, p)
                
                results.append(stats)
        
        results_df = pd.DataFrame(results)
        print(f"   ✓ Calculated distribution for {len(results_df)} months")
        
        return results_df
    
    def save_distributions(self, hourly_dist, daily_dist, monthly_dist, site_name, data_type, folder_name, file_prefix):
        """
        Generic function to save distribution files
        """
        # Create distribution folder
        dist_path = self.base_path / site_name / folder_name / 'forecast' / 'distribution'
        dist_path.mkdir(parents=True, exist_ok=True)
        
        print(f"\n📊 Saving {data_type.upper()} distribution files...")
        
        # Determine float format based on data type
        float_fmt = '%.3f' if 'generation' in data_type.lower() else '%.2f'
        
        # Hourly distribution
        hourly_dist_file = f"{site_name}_{file_prefix}_hourly_distribution.csv"
        cols_hourly = ['datetime_label', 'year', 'month', 'day', 'hour', 'mean', 'std_dev'] + \
                     [f'p{p}' for p in self.percentiles] + ['min', 'max', 'count']
        
        hourly_dist[cols_hourly].to_csv(dist_path / hourly_dist_file, index=False, float_format=float_fmt)
        print(f"   ✓ {hourly_dist_file}")
        
        # Daily distribution
        daily_dist_file = f"{site_name}_{file_prefix}_daily_distribution.csv"
        cols_daily = ['date_label', 'year', 'month', 'day', 'mean', 'std_dev'] + \
                    [f'p{p}' for p in self.percentiles] + ['min', 'max', 'count']
        
        daily_dist[cols_daily].to_csv(dist_path / daily_dist_file, index=False, float_format=float_fmt)
        print(f"   ✓ {daily_dist_file}")
        
        # Monthly distribution
        monthly_dist_file = f"{site_name}_{file_prefix}_monthly_distribution.csv"
        cols_monthly = ['month_name', 'year', 'month', 'mean', 'std_dev'] + \
                      [f'p{p}' for p in self.percentiles] + ['min', 'max', 'count']
        
        monthly_dist[cols_monthly].to_csv(dist_path / monthly_dist_file, index=False, float_format=float_fmt)
        print(f"   ✓ {monthly_dist_file}")
        
        print(f"\n📁 {data_type} distribution files saved in:")
        print(f"   • Renewable Portfolio LLC/{site_name}/{folder_name}/forecast/distribution/")
    
    def print_distribution_sample(self, hourly_dist, daily_dist, monthly_dist, data_type='generation'):
        """
        Print sample results from distributions
        """
        print("\n" + "="*40)
        print(f"SAMPLE {data_type.upper()} DISTRIBUTION RESULTS")
        print("="*40)
        
        # Sample hourly distribution
        if not hourly_dist.empty:
            sample_idx = len(hourly_dist) // 2  # Middle of the period
            if sample_idx < len(hourly_dist):
                row = hourly_dist.iloc[sample_idx]
                print(f"\n⚡ Hourly Sample - {row['datetime_label']}:")
                if data_type == 'generation':
                    print(f"   Mean: {row['mean']:.2f} MW")
                    print(f"   P10: {row['p10']:.2f} MW | P50: {row['p50']:.2f} MW | P90: {row['p90']:.2f} MW")
                elif 'revenue' in data_type.lower():
                    print(f"   Mean: ${row['mean']:,.2f}")
                    print(f"   P10: ${row['p10']:,.2f} | P50: ${row['p50']:,.2f} | P90: ${row['p90']:,.2f}")
                else:
                    print(f"   Mean: ${row['mean']:.2f}")
                    print(f"   P10: ${row['p10']:.2f} | P50: ${row['p50']:.2f} | P90: ${row['p90']:.2f}")
                print(f"   Based on {int(row['count'])} scenarios")
        
        # Sample daily distribution
        if not daily_dist.empty:
            sample_idx = len(daily_dist) // 2
            if sample_idx < len(daily_dist):
                row = daily_dist.iloc[sample_idx]
                print(f"\n📅 Daily Sample - {row['date_label']}:")
                if data_type == 'generation':
                    print(f"   Mean: {row['mean']:.2f} MWh")
                    print(f"   P10: {row['p10']:.2f} MWh | P50: {row['p50']:.2f} MWh | P90: {row['p90']:.2f} MWh")
                elif 'revenue' in data_type.lower():
                    print(f"   Mean: ${row['mean']:,.2f}")
                    print(f"   P10: ${row['p10']:,.2f} | P50: ${row['p50']:,.2f} | P90: ${row['p90']:,.2f}")
                else:
                    print(f"   Mean: ${row['mean']:.2f}")
                    print(f"   P10: ${row['p10']:.2f} | P50: ${row['p50']:.2f} | P90: ${row['p90']:.2f}")
                print(f"   Based on {int(row['count'])} scenarios")
    
    def process_distributions(self, site_name, data_type, folder_name, file_prefix):
        """
        Generic function to process distributions for any data type
        """
        print("\n" + "-"*40)
        print(f"PROCESSING {data_type.upper()} DISTRIBUTIONS")
        print("-"*40)
        
        # Define paths to timeseries files
        ts_path = self.base_path / site_name / folder_name / 'forecast' / 'timeseries'
        
        hourly_ts_file = ts_path / f"{site_name}_{file_prefix}_hourly_timeseries.csv"
        daily_ts_file = ts_path / f"{site_name}_{file_prefix}_daily_timeseries.csv"
        monthly_ts_file = ts_path / f"{site_name}_{file_prefix}_monthly_timeseries.csv"
        
        # Check if files exist
        if not all([hourly_ts_file.exists(), daily_ts_file.exists(), monthly_ts_file.exists()]):
            print(f"❌ Missing {data_type} timeseries files for {site_name}")
            return False
        
        # Calculate distributions
        hourly_dist = self.calculate_hourly_distribution(hourly_ts_file, data_type)
        daily_dist = self.calculate_daily_distribution(daily_ts_file, data_type)
        monthly_dist = self.calculate_monthly_distribution(monthly_ts_file, data_type)
        
        # Print sample results
        self.print_distribution_sample(hourly_dist, daily_dist, monthly_dist, data_type)
        
        # Save distribution files
        self.save_distributions(hourly_dist, daily_dist, monthly_dist, site_name, data_type, folder_name, file_prefix)
        
        # Print revenue summary if processing revenue
        if 'revenue' in data_type.lower():
            self.print_revenue_summary(monthly_dist, data_type)
        
        return True
    
    def print_revenue_summary(self, monthly_dist, revenue_type):
        """
        Print summary of revenue results
        """
        print("\n" + "="*50)
        print(f"{revenue_type.upper()} SUMMARY")
        print("="*50)
        
        # Annual revenue estimate (sum of monthly means)
        annual_revenue_mean = monthly_dist['mean'].sum()
        annual_revenue_p10 = monthly_dist['p10'].sum()
        annual_revenue_p90 = monthly_dist['p90'].sum()
        
        print(f"\n💰 Estimated Annual Revenue:")
        print(f"   Mean: ${annual_revenue_mean:,.2f}")
        print(f"   P10:  ${annual_revenue_p10:,.2f}")
        print(f"   P90:  ${annual_revenue_p90:,.2f}")
        
        # Monthly breakdown - top 3 months
        top_months = monthly_dist.nlargest(3, 'mean')
        print(f"\n📊 Top Revenue Months:")
        for _, row in top_months.iterrows():
            print(f"   {row['month_name']:9s}: Mean ${row['mean']:,.2f} (P10: ${row['p10']:,.2f}, P90: ${row['p90']:,.2f})")
    
    def check_generation_metadata(self, site_name):
        """
        Check which bootstrap was used for generation data
        """
        metadata_file = self.base_path / site_name / 'Generation' / 'generation_metadata.json'
        if metadata_file.exists():
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
                return metadata.get('bootstrap_type', 'unknown')
        return 'unknown'
    
    def process_single_site(self, site_name):
        """
        Process all distributions for a single site
        """
        print(f"\n{'='*60}")
        print(f"Processing distributions for: {site_name}")
        print(f"{'='*60}")
        
        # Check which bootstrap was used for generation
        gen_bootstrap = self.check_generation_metadata(site_name)
        if gen_bootstrap != 'unknown':
            gen_pipeline = 'DA' if gen_bootstrap == 'da' else 'RT'
            print(f"\n📌 Note: Generation data was created using {gen_pipeline} bootstrap")
            if gen_bootstrap != self.pipeline_type:
                print(f"   ⚠️  Warning: Current pipeline ({self.pipeline_name_short}) differs from generation bootstrap ({gen_pipeline})")
        
        success = True
        
        try:
            # Process Generation distributions
            if not self.process_distributions(site_name, 'Generation', 'Generation', 'generation'):
                success = False
            
            # Process Price distributions (pipeline-specific)
            if not self.process_distributions(site_name, f'{self.pipeline_name} Price', self.price_folder, self.price_prefix):
                success = False
            
            # Process Revenue distributions (pipeline-specific)
            if not self.process_distributions(site_name, f'{self.pipeline_name} Revenue', self.revenue_folder, self.revenue_prefix):
                success = False
            
            return success
            
        except Exception as e:
            print(f"\n❌ Error processing {site_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            return False
    
    def run_calculator(self):
        """
        Main function to run the distribution calculator
        """
        print(f"\n🌟 {self.pipeline_name} Distribution Calculator - Phase 2")
        print(f"   Calculate distributions for Generation, Price, and Revenue")
        print(f"   Using compressed timeseries from Phases 1a, 1b, and 1c")
        print("="*60)
        
        # Get site selection
        site_selection = self.get_site_selection()
        if not site_selection:
            return
        
        site_selection, sites_to_process = site_selection
        
        # Process based on selection
        if site_selection == 'ALL_SITES':
            # Process all sites
            print("\n" + "="*60)
            print("🚀 PROCESSING ALL SITES")
            print("="*60)
            
            successful = 0
            failed = 0
            
            for i, site_name in enumerate(sites_to_process, 1):
                print(f"\n[{i}/{len(sites_to_process)}] Processing {site_name}...")
                
                if self.process_single_site(site_name):
                    successful += 1
                else:
                    failed += 1
            
            # Summary
            print("\n" + "="*60)
            print("✨ PHASE 2 COMPLETE!")
            print("="*60)
            print(f"\n📊 Summary:")
            print(f"   ✅ Successfully processed: {successful} sites")
            if failed > 0:
                print(f"   ❌ Failed: {failed} sites")
            
            print(f"\n📁 Distribution files created for:")
            print(f"   • Generation/forecast/distribution/")
            print(f"   • {self.price_folder}/forecast/distribution/")
            print(f"   • {self.revenue_folder}/forecast/distribution/")
            
        else:
            # Process single site
            if self.process_single_site(site_selection):
                print("\n" + "="*60)
                print("✨ PHASE 2 COMPLETE!")
                print("="*60)
                print(f"\nDistributions calculated for {site_selection}")
                
                print(f"\n📁 Files saved in:")
                print(f"   • {site_selection}/Generation/forecast/distribution/")
                print(f"   • {site_selection}/{self.price_folder}/forecast/distribution/")
                print(f"   • {site_selection}/{self.revenue_folder}/forecast/distribution/")
        
        print(f"\n✅ {self.pipeline_name} Pipeline Complete!")
        print(f"\n📌 Pipeline Summary:")
        print(f"   Phase 0: Bootstrap selections")
        print(f"   Phase 1a: Generation timeseries")
        print(f"   Phase 1b: Price timeseries (compressed)")
        print(f"   Phase 1c: Revenue timeseries (compressed)")
        print(f"   Phase 2: All distributions ✓")
        
        # Ask if user wants to process another site
        another = input("\n🔄 Calculate distributions for another site? (y/n): ").strip().lower()
        if another == 'y':
            self.run_calculator()

# Example usage
if __name__ == "__main__":
    # Ask user which pipeline to run
    print("\n📊 Distribution Calculator")
    print("Select pipeline type:")
    print("1. Day-Ahead (DA)")
    print("2. Real-Time (RT)")
    
    while True:
        choice = input("\nEnter your choice (1 or 2): ").strip()
        if choice == '1':
            calc = DistributionCalculator(pipeline_type='da')
            break
        elif choice == '2':
            calc = DistributionCalculator(pipeline_type='rt')
            break
        else:
            print("❌ Invalid choice! Please enter 1 or 2.")
    
    calc.run_calculator()


📊 Distribution Calculator
Select pipeline type:
1. Day-Ahead (DA)
2. Real-Time (RT)

🌟 REAL-TIME Distribution Calculator - Phase 2
   Calculate distributions for Generation, Price, and Revenue
   Using compressed timeseries from Phases 1a, 1b, and 1c

REAL-TIME DISTRIBUTION CALCULATOR - SITE SELECTION

✅ Found 7 sites with complete REAL-TIME data
   (Generation, Price, and Revenue timeseries)

Available options:
0. ALL SITES (Process all ready sites)
1. Albemarle_Beach_Solar
2. Blue_Wing_Solar_Energy_Generator
3. Lamesa_Solar
4. Midway_Solar_Farm_III
5. Misae_Solar
6. Mount_Signal_Solar_Farm_II
7. RE_Mustang_LLC

🚀 PROCESSING ALL SITES

[1/7] Processing Albemarle_Beach_Solar...

Processing distributions for: Albemarle_Beach_Solar

📌 Note: Generation data was created using DA bootstrap
   ⚠️  Warning: Current pipeline (RT) differs from generation bootstrap (DA)

----------------------------------------
PROCESSING GENERATION DISTRIBUTIONS
----------------------------------------

⚡ Calc